In [1]:
import os, random, time
import numpy as np, numpy.linalg as la, scipy as sp, matplotlib.pyplot as plt, matplotlib.ticker as ticker, matplotlib.animation as animation, scipy.constants as ct, scipy.optimize as sciopt
from IPython.display import HTML
from IPython.display import set_matplotlib_formats
%matplotlib tk
plt.rcParams.update({'font.size': 20})
set_matplotlib_formats('pdf', 'svg')
from collections import defaultdict
from math import *
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d, InterpolatedUnivariateSpline, RegularGridInterpolator, RectBivariateSpline
from scipy.signal import savgol_filter, find_peaks, hann, hilbert
from scipy.optimize import minimize, curve_fit
from scipy.misc import derivative
from numpy.fft import rfft, rfftfreq, irfft

import PyLTSpice
from PyLTSpice.LTSpice_RawRead import LTSpiceRawRead
from PyLTSpice.LTSpiceBatch import LTCommander

import VoltageGeneration

Found Numpy. WIll be used for storing data


# Initialize Trap

**To do**: 
- clean this up. find a way to avoid explicitly defining both SLT and MaxBeta configurations for ITVG. would like to set the ordering without this num_to_electorde_name() function ✅
- see if there's a way to instantiate the trap without specifying the ion species. would be nice to just have to set electrode geometry, since after all the trap is agnostic to the ion

In [2]:
#### MaxBeta
reload_vg = True
amu = ct.m_u
e = ct.e
Ca = 40*amu
Sr = 88*amu

electrode_ordering = [
    'S21', 'N1', 'S20', 'N2', 'S19', 'N3', 'S14',
    'N4', 'S13', 'N8', 'S12', 'N9', 'S11', 'N10',
    'S10', 'N11', 'S9', 'N12', 'S8', 'N13', 'S4',
    'N14', 'S3', 'N19', 'S2', 'N20', 'S1', 'N21',
    'S25', 'S24', 'S23', 'S22'
]

path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/GridFiles'
name = 'MaxBeta_Production_Potential'

electrode_grouping = [[i+1] for i in range(len(electrode_ordering))]
# electrode_grouping = [[i+1] for i in range(len(electrode_ordering)) if i not in [18,19,22,23]]
# electrode_grouping.extend([[19,20],[23,24]])
print(electrode_grouping)

d_cons = []
custom_fit = ['x','y','z','xx','xy','xz','yy','yz','zz','yyy','yyyy','yyyyy','yyyyyy']

start_time = time.time()
if reload_vg:
    vg = VoltageGeneration.VoltageGeneration(path, name, electrode_grouping=electrode_grouping, fit_ranges=[5e-6, 10e-6, 5e-6], rf_electrodes=[33], m=Ca, rf_omega=2.*np.pi*45.937e6, order=4, f_cons=d_cons, v_rf=[72])
print("Imported grid files in {}".format(time.time() - start_time))


[[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]
Importing electrode 9
Importing electrode 8
Importing electrode 23
Importing electrode 22
Importing electrode 20
Importing electrode 21
Importing electrode 19
Importing electrode 31
Importing electrode 25
Importing electrode 24
Importing electrode 30
Importing electrode 18
Importing electrode 26
Importing electrode 32
Importing electrode 33
Importing electrode 27
Importing electrode 16
Importing electrode 17
Importing electrode 15
Importing electrode 29
Importing electrode 28
Importing electrode 14
Importing electrode 10
Importing electrode 11
Importing electrode 13
Importing electrode 12
Importing electrode 1
Importing electrode 3
Importing electrode 2
Importing electrode 6
Importing electrode 7
Importing electrode 5
Importing electrode 4
Potentials imported. (dev version)
Imported grid files in 93.94030

In [ ]:
#### SLT
reload_vg = False
amu = ct.m_u
e = ct.e
Ca = 40*amu
Sr = 88*amu

electrode_ordering = [
    "S6", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14",
    "S24", "S25",
    "N6", "N7", "N8", "N9", "N10", "N11", "N12", "N13", "N14",
] ## Ordering for the SLT configuration

path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/GridFiles'
name = 'AngleTrap_Skinny_Potential_reduced'

electrode_grouping = [[i+1] for i in range(len(electrode_ordering))]
d_cons = []

start_time = time.time()
if reload_vg:
    vg = VoltageGeneration.VoltageGeneration(path, name, electrode_grouping=electrode_grouping, fit_ranges=[5e-6, 10e-6, 5e-6], rf_electrodes=[21], m=Ca, rf_omega=2.*np.pi*45.247e6, order=4, f_cons=d_cons, v_rf=[50])
print("Imported grid files in {}".format(time.time() - start_time))


# Set Ideal Motion Protocol

**Collaboration with Gonzalo**

In [3]:
def qtanhN(t, tspan, N=5, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T = tf-t0
    if (t-t0) < T:
        return y0 + max((yf-y0)/2*(np.tanh(N*(2*(t-t0)-T)/T)+np.tanh(N))/np.tanh(N), 0)
    else:
        return yf
    
def qsin(t, tspan, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T = tf-t0
    if t < t0:
        return y0
    elif (t-t0) < T:
        return y0 + max((yf-y0)/2*(1-np.cos(pi*(t-t0)/T)), 0)
    else:
        return yf

def qsta(t, tspan, qspan=(0, 120), f=1):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*((1/(2*pi*f)**2)*(60*s - 180*s**2 + 120*s**3) + 10*s**3 - 15*s**4 + 6*s**5)
    else:
        return yf
    
def qsta2(t, tspan, qspan=(0, 120), f=1):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*((1/(2*pi*f)**2)*(2520*s**3-12600*s**4+22680*s**5-17640*s**6+5040*s**7)
                             + 126*s**5-420*s**6+540*s**7-315*s**8+70*s**9)
    else:
        return yf
    
def qtrig(t, tspan, qspan=(0, 120), f=1.2e6, mu=1):
    ''' args should be f (axialfreq in Hz), mu (mass ratio)'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti, d = tf-t0, t-t0, yf-y0
    s = ti/T
    Op, On = 2*np.pi*f*(1+1/mu+(1-1/mu+1/mu**2)**0.5)**0.5, 2*np.pi*f*(1+1/mu-(1-1/mu+1/mu**2)**0.5)**0.5
    b3 = -49*((T*Op)**2 - (5*np.pi)**2)*((T*On)**2 - (5*np.pi)**2)/(2048*(T*T*On*Op)**2)
    b4 = 5*((T*Op)**2 - (7*np.pi)**2)*((T*On)**2 - (7*np.pi)**2)/(2048*(T*T*On*Op)**2)
    
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + d*(0.5 + (-9/16+2*b3+5*b4)*np.cos(np.pi*s) + 1/16*(1-48*b3-96*b4)*np.cos(3*np.pi*s) + b3*np.cos(5*np.pi*s) + b4*np.cos(7*np.pi*s))
    else:
        return yf
    
def d_t(t, tspan, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*(10*s**3 - 15*s**4 + 6*s**5)
    else:
        return yf

def get_freq_from_alpha(a, m_=40*ct.u):
    ''' make sure alpha is in V/um^2'''
    return np.sign(a)*(2*np.abs(a)*1e12*ct.e/m_)**0.5/(2*pi)

def get_alpha_from_freq(f, m_=40*ct.u):
    ''' returns alpha with units of V/m^2'''
    return (2*pi*f)**2/(2*ct.e/m_)

def p_derivative(func, ms=1e-11, var=1, point=[], order=1):
    ''' implements a partial derivative '''
    args = point[:]
    def wraps(x):
        args[var] = x
        return func(*args)
    return derivative(wraps, point[var], dx = ms, n=order)

def gaussian(x, mu, sig):
    return np.exp(-(x-mu)*(x-mu)/(2*sig*sig))

## Transport

In [26]:
f0 = 4e6 # Axial frequency [Hz]
T0 = 1.96e-6 # Transport time [s]
move = (186.5, 231.5) # Start and End positions [um]
a0 = get_alpha_from_freq(f0, m_=Ca)*1e-12 # Specify ion mass. [Units are V/m^2, but ITVG uses um as the length scale]

t_ideal = np.linspace(0, T0, 100)
q_ideal = np.array([qsta2(t, (0, T0), qspan=move, f=T0*f0) for t in t_ideal])
# q_ideal = np.linspace(move[0], move[1], len(t_ideal))
a_ideal = np.repeat(a0, len(t_ideal))
q_ideal_func = interp1d(np.append(t_ideal, 1), np.append(q_ideal, q_ideal[-1]))
a_ideal_func = interp1d(np.append(t_ideal, 1), np.append(a_ideal, a_ideal[-1]))

# plt.plot(t_ideal*1e6, q_ideal)
# plt.xlabel('time (us)')
# plt.ylabel('position (um)')

In [ ]:
f0 = 0.4e6 # Axial frequency [Hz]
a0 = get_alpha_from_freq(f0, m_=Ca)*1e-12 # Specify ion mass. [Units are V/m^2, but ITVG uses um as the length scale]

T0 = 10e-6
t_ideal = np.array(np.linspace(0, T0, 10))
q_ideal = np.array([-186.5]*10)
# a_ideal = np.repeat(a0, len(t_ideal))
a_ideal = [get_alpha_from_freq(f, m_=Ca)*1e-12 for f in np.linspace(0.4e6, 4e6, 10)]


In [ ]:
#### bang bang (bang)
f0 = 2.5e6
a0 = get_alpha_from_freq(f0, m_=Ca)*1e-12
dt = 20e-6

t_ideal = np.array([0, 1, 2, 3])*dt
q_ideal = np.array([0, 40, 80, 120])
a_ideal = np.repeat(a0, len(t_ideal))
T0 = t_ideal[-1]

plt.plot(t_ideal*1e6, q_ideal, marker='.')

## Splitting

**STA**

In [ ]:
f0 = 1.2e6
T0 = 300e-6
q0 = 186.5 # center of split [um]
mass = Ca

eps0 = ct.epsilon_0
K = ct.e/(4*pi*eps0)
_a10, _a11 = -780, 127
a0 = get_alpha_from_freq(f0, m_=mass)
l0 = 2*a0/mass*np.array([1, 3]) ## (-, +)
lf = 2*a0/mass*np.array([1, 1.002])
gn, gp = (l0/lf)**0.25
rho_p_t = lambda s, a10, a11: 1-(126*(1-gp) + a10 + 5*a11)*s**5 + (420*(1-gp) + 5*a10 + 24*a11)*s**6 - (540*(1-gp) + 10*a10 + 45*a11)*s**7 + (315*(1-gp) + 10*a10 + 40*a11)*s**8 - (70*(1-gp) + 5*a10 + 15*a11)*s**9 + a10*s**10 + a11*s**11

t_ideal = np.linspace(0, T0, 100) ## us
s_view = t_ideal/t_ideal[-1]
rho_p = rho_p_t(s_view, _a10, _a11)
d2rho_pdt2 = np.array([p_derivative(rho_p_t, ms=1e-5, var=0, point=[s, _a10, _a11], order=2) for s in s_view])
lp_t = l0[1]*rho_p**(-4) - d2rho_pdt2/rho_p
ln_t = np.full(len(s_view), l0[0])
d_ideal = (4*K/(mass*(lp_t-ln_t)))**(1/3) ## SI units [m]
a_ideal = (mass/8*(3*lp_t - 5*ln_t)) ## SI units [V/m^2]
b_ideal = (2*K*(d_ideal)**(-5) - 2*a_ideal*(d_ideal)**(-2)) ## SI units [V/m^4]

w_ideal = np.array([(p_derivative(phi, point=[t, d, a, b, mass], order=2, ms=1e-6)*ct.e/mass)**0.5 for t, d, a, b in zip(t_ideal, d_ideal, a_ideal, b_ideal)])
w2_ideal = np.array([(ct.e/mass*(2*a + 3*b*d**2))**0.5 for a, b, d in zip(a_ideal, b_ideal, d_ideal)])
w2_stretch_ideal = w2_ideal*(1+4*ct.e*K/(mass*(w2_ideal)**2*(1e-6*d_ideal)**3))**0.5

fig, ax = plt.subplots()
ax2 = ax.twinx()
ax.plot(t_ideal, a_ideal/a_ideal[0], label='α')
ax.plot(t_ideal, b_ideal/np.max(b_ideal), label='β')
ax.legend()
# ax2.plot(t_ideal, w_ideal/(2*pi*1e6), 'r--', label='ω, numerical')
ax2.plot(t_ideal, w2_ideal/(2*pi*1e6), 'r', label='ω, Home (2006)')
ax2.plot(t_ideal, w2_stretch_ideal/(2*pi*1e6), 'r--', label='ω+, Home (2006)')
ax2.set_ylabel('Mhz', color='r')
ax2.tick_params(axis='y', colors='red')
ax2.legend()
# d_ideal_func = interp1d(np.append(t_ideal, 1), np.append(d_ideal*1e6, d_ideal[1]*1e6))

a_ideal *= 1e-12 ## convert to um units
b_ideal *= 1e-24

plt.figure('axial')
plt.plot(t_ideal, d_ideal)


**Kaufmann**

In [ ]:
bmax = 1.8e-7
T0 = 300e-6
q0 = 186.5 # center of split [um]
eps0 = ct.epsilon_0
K = ct.e/(4*pi*eps0)
Ca = 40*ct.m_u
Sr = 88*ct.m_u
Ba = 133*ct.m_u
d_CP = 0.5*(2*K/(bmax*1e24))**0.2
mass = Ca

t_ideal, dx = np.linspace(0, T0, 100, retstep=True) ## us
d_ideal = np.array([d_t(t, (t_ideal[0], t_ideal[-1]), qspan=(2.5, 22.5)) for t in t_ideal]) ## um (half the separation)
t_CP = interp1d(d_ideal, t_ideal).__call__(d_CP*1e6)
print('d_CP: ', d_CP*1e6, 't_CP: ', t_CP)

b_ideal = np.array([bmax*qsta2(t, (0, t_CP), qspan=(0, 1), f=5) if t<t_CP else bmax*(1-0.9*qsta2(t-t_CP, (t_CP*0, t_CP*2), qspan=(0, 1), f=5)) for t in t_ideal]) ## V/um^4
a_ideal = 1e-12*np.array([K*d**(-3) - 0.5*b*d**2 for b, d in zip(b_ideal*1e24, 2*d_ideal*1e-6)]) ## V/um^2
w_ideal = np.array([(p_derivative(phi, point=[t, d, a, b, mass], order=2)*ct.e/mass)**0.5 for t, d, a, b in zip(t_ideal*1e-6, d_ideal*1e-6, a_ideal*1e12, b_ideal*1e24)])
w2_ideal = np.array([(ct.e/mass*(2*a + 3*b*d**2))**0.5 for a, b, d in zip(a_ideal*1e12, b_ideal*1e24, d_ideal*2e-6)])
w2_stretch_ideal = w2_ideal*(1+4*ct.e*K/(mass*(w2_ideal)**2*(2e-6*d_ideal)**3))**0.5

fig, ax = plt.subplots()
ax2 = ax.twinx()
ax.plot(t_ideal, a_ideal/a_ideal[0], label='α')
ax.plot(t_ideal, b_ideal/np.max(b_ideal), label='β')
ax.legend()
ax2.plot(t_ideal, w_ideal/(2*pi*1e6), 'r', label='ω, numerical')
ax2.plot(t_ideal, w2_ideal/(2*pi*1e6), 'r--', label='ω, Home (2006)')
# ax2.plot(t_ideal, w2_stretch_ideal/(2*pi*1e6), ls='--', label='ω+, Home (2006)')
ax2.set_ylabel('Mhz', color='r')
ax2.tick_params(axis='y', colors='red')
ax2.legend()


**Kaufmann with constant frequency**

In [9]:
f0 = 1.2e6
T0 = 300e-6
q0 = 186.5 # center of split [um]

eps0 = ct.epsilon_0
K = ct.e/(4*pi*eps0)
Ca = 40*ct.m_u
Sr = 88*ct.m_u
Ba = 133*ct.m_u
mass = Ca
bmax = ((2*np.pi*f0)**2*mass/(2*ct.e)/(2*K)**0.4)**(5/3)
d_CP = 0.5*(2*K/(bmax))**0.2 ## half the separation at CP

t_ideal, dx = np.linspace(0, T0, 100, retstep=True) ## us
a0 = (2*pi*f0)**2*mass/(2*ct.e)
d0 = (K/a0)**(1/3)
print(d0/2*1e6)
d_ideal = np.array([d_t(t, (t_ideal[0], t_ideal[-1]), qspan=(d0/2*1e6, 45)) for t in t_ideal]) ## um (half the separation)
# d_ideal = np.array([d_t(t, (t_ideal[0], t_ideal[50]), qspan=(d0/2e-6, d_CP/1e-6)) for t in t_ideal[:50]]+
#         [d_t(t, (t_ideal[50], t_ideal[-1]), qspan=(d_CP/1e-6, 45)) for t in t_ideal[50:]]) ## um (half the separation)
t_CP = interp1d(d_ideal, t_ideal).__call__(d_CP*1e6)
print('d_CP: ', d_CP*1e6, 't_CP: ', t_CP)

ln_ideal = np.repeat(2*pi*f0/sqrt(ct.e), len(t_ideal))
lp_ideal = np.sqrt(4*K/mass*(d_ideal*2e-6)**(-3) + ln_ideal**2)
a_ideal = mass/8*(3*lp_ideal**2 - 5*ln_ideal**2) ## V/m^2
b_ideal = 2*K/(d_ideal*2e-6)**5 - 2*a_ideal/(d_ideal*2e-6)**2 ## V/m^4

w_ideal = np.array([(p_derivative(phi, point=[t, d, a, b, mass], order=2)*ct.e/mass)**0.5 for t, d, a, b in zip(t_ideal, d_ideal*2e-6, a_ideal, b_ideal)])
w2_ideal = np.array([(ct.e/mass*(2*a + 3*b*d**2))**0.5 for a, b, d in zip(a_ideal, b_ideal, d_ideal*2e-6)])
w2_stretch_ideal = w2_ideal*(1+4*ct.e*K/(mass*(w2_ideal)**2*(d_ideal*2e-6)**3))**0.5

fig, ax = plt.subplots()
ax2 = ax.twinx()
ax.plot(t_ideal, a_ideal/a_ideal[0], label='α')
ax.plot(t_ideal, b_ideal/np.max(b_ideal), label='β')
ax.legend()
# ax2.plot(t_ideal, w_ideal/(2*pi*1e6), 'r--', label='ω, numerical')
ax2.plot(t_ideal, w2_ideal/(2*pi*1e6), 'r', label='ω, Home (2006)')
ax2.plot(t_ideal, w2_stretch_ideal/(2*pi*1e6), 'r--', label='ω+, Home (2006)')
ax2.set_ylabel('Mhz', color='r')
ax2.tick_params(axis='y', colors='red')
ax2.legend()

a_ideal *= 1e-12
b_ideal *= 1e-24


2.481172513909303
d_CP:  3.126081478694905 t_CP:  3.672480088545283e-05


# Find Voltages
**Aka, backwards ITVG**

In [7]:
def rotate_coeffs(theta, xx, xz, zz):
    """
    Find the coefficients $C'_{xx}, C'_{xz}, C'_{zz}$ of the potential
    after a rotation in the zx-plane by `theta` given the unprimed values,
    as described above.
    
    `theta` specified in radians.
    """
    sin = np.sin
    cos = np.cos
    
    xxnew = xx*cos(theta)**2 + zz*sin(theta)**2 + 0.5*xz*sin(2*theta)
    xznew = xz*cos(2*theta)  + (zz-xx)*sin(2*theta)
    zznew = zz*cos(theta)**2 + xx*sin(theta)**2 - 0.5*xz*sin(2*theta)
    
    return [xxnew, xznew, zznew]

def dcTiltAngle(vg, R, theta_targ, yy, rfEls, delta=3, debug=False):
    # expansion coefficients of the pseudopotential near the trapping point `R`
    # THIS FUNCTION ASSUMES ONE RF ELECTRODE!
    rf_coeffs = vg.printCoefficients(R, rfEls, ('zz', 'xx', 'xz', 'yy', 'xy', 'yz'), printing=False)[0]
    [rfzz, rfxx, rfxz, rfyy, rfxy, rfyz] = rf_coeffs[:,0]
    
    def tiltError(theta_tilt, debug=False):
        """
        Error in the tilt angle of the full potential, as a function of the dc tilt.
        """
        
        zz = delta*yy
        xx = -(1+delta)*yy # delta ensures Laplace (div phi = 0) is satisfied, but adjusts the principal axes of the xz ellipse
        xz = 0
        
        # rotate!
        [xxnew, xznew, zznew] = rotate_coeffs(theta_tilt, xx, xz, zz)

        if debug:
            print (np.degrees(theta_tilt))
            print (np.degrees(np.arctan((xznew+rfxz)/((rfzz+zznew) - (rfxx+xxnew))))/2)
        
        return (
            np.tan(2*theta_targ) - (xznew+rfxz)/((rfzz+zznew) - (rfxx+xxnew))
        )**2
    
    if debug:
        pass
#         print (np.degrees(theta_targ))

#         fig, ax = plt.subplots()
#         angle_space = np.linspace(0, np.pi/2, 91)
#         errors = [tiltError(theta) for theta in angle_space]
#         ax.plot(np.degrees(angle_space), errors)
#         ax.set_ylim(0,1)
    
    return sciopt.minimize_scalar(tiltError).x

def tiltWithPseudo(vg, R, theta_targ, yy, rfEls, delta=3, debug=False):
    th = dcTiltAngle(vg, R, theta_targ, yy, rfEls, delta=delta, debug=debug)
    
    if debug:
        print ("Tilting dc to {:.2f} deg".format(np.degrees(th)))
    
    zz = delta*yy
    xx = -(1+delta)*yy
    xz = 0
    return rotate_coeffs(th, xx, xz, zz)


In [8]:
MB_intervals = {14:[0,74], 13:[74,119], 12:[119,164], 11:[164,209], 10:[209,254], 9:[254,299], 8:[299,373],
               7:[373,447], 6:[447,492], 5:[492,537], 4:[537,582], 3:[582,627], 2:[627,672], 1:[672,746],
               15:[-74,0], 16:[-119,-74], 17:[-164,-119], 18:[-209,-164], 19:[-254,-209,], 20:[-299,-254], 21:[-373,-299],
               22:[-447,-373], 23:[-492,-447], 24:[-537,-492], 25:[-582,-537], 26:[-627,-582], 27:[-672,-627], 28:[-746,-672]} # MaxBeta electrode intervals

SLT_intervals = {1:[], 2:[-420,-300], 3:[-300,-180], 4:[-180,-60], 5:[-60,60], 6:[60,180], 7:[180,300], 8:[300,420], 9:[], 12:[], 20:[],
                19:[-420,-300], 18:[-300,-180], 17:[-180,-60], 16:[-60,60], 15:[60,180], 14:[180,300], 13:[300,420]}

def getOverlap(a, b):
    return max(0, min(a[1], b[1]) - max(a[0], b[0]))

def get_constrained_voltages(q, tilt_el={'S25': 12.930878042522904, 'S24': -24.036409224366135, 'S23': -39.5, 'S22': 27.90731149920946}):
    tilt_el_num = [29,30,31,32]

    if q<=-604.5:
        volt = [(p[0]-1, qsta2(q, (74, 119), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[25,26,27,28]]
    elif q<=-559.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[24,25,26,27]]
    elif q<=-514.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[23,24,25,26]]
    elif q<=-469.5:
        volt = [(p[0]-1, qsta2(q, (254, 299), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[22,23,24,25]]
    elif q<=-410:
        volt = [(p[0]-1, qsta2(q, (254, 299), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[21,22,23,24]]
    elif q<=-336:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[20,21,22,23]]
    elif q<=-276.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[19,20,21,22]]
    elif q<=-231.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[18,19,20,21]]
    elif q<=-186.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[17,18,19,20]]
    elif q<=-141.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[16,17,18,19]]
    elif q<=-86:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[15,16,17,18]]
    elif q<=-21.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[14,15,16,17]]  
    elif q<=21.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[13,14,15,16]]
    elif q<=86:
        volt = [(p[0]-1, qsta2(q, (74, 119), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[12,13,14,15]]
    elif q<=141.5:
        volt = [(p[0]-1, qsta2(q, (74, 119), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[11,12,13,14]]
    elif q<=186.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[10,11,12,13]]
    elif q<=231.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[9,10,11,12]]
    elif q<=276.5:
        volt = [(p[0]-1, qsta2(q, (254, 299), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[8,9,10,11]]
    elif q<=336:
        volt = [(p[0]-1, qsta2(q, (254, 299), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[7,8,9,10]]
    elif q<=410:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[6,7,8,9]]
    elif q<=469.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[5,6,7,8]]
    elif q<=514.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[4,5,6,7]]
    elif q<=559.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[3,4,5,6]]
    elif q<=604.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[2,3,4,5]]
    else:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[1,2,3,4]]
        
    volt += [(p-1, tilt_el[electrode_ordering[p-1]]) for p in tilt_el_num]
    return volt


In [ ]:
allvoltages_hackathon = [{'S1': 21.000,'S2': 21.000,'S3': 21.000,'S4': 21.000,'S8': 1.473,'S9': 1.659,'S10': 21.000,'S11': 21.000, 'S12': 21.000, 'S13': 21.000, 'S14': 21.000, 'S19': 21.000, 'S20': 21.000, 'S21': 21.000, 'S22': 39.900, 'S23': -39.900, 'S24': 39.900, 'S25': -36.834, 'N1': 21.000, 'N2': 21.000, 'N3': 21.000, 'N4': 21.000, 'N8': 21.000, 'N9': 21.000, 'N10': 21.000, 'N11': 1.103, 'N12': 2.331, 'N13': 2.254, 'N14': 21.000, 'N19': 21.000, 'N20': 21.000,'N21': 21.000}
]
''' taylor fit to these voltages and overlay an SLT potential at the same frequecy
try to '''


## Transport

In [27]:
tilt_angle = np.radians(10) # specify tilt [rad]
volt_gen = vg
bounds = (-39.5, 39.5)
norm = np.full((1, 1), 1) # normalization factor to pull out certain coefficients
# norm[[1,4]] = 5e1 # y and yy coefficients
# norm[[7,8]] = 1e2
volt_gen.set_electrode_grouping(electrode_grouping) # sets previous voltages to None

allvoltages = []
for q, a, i in zip(q_ideal, a_ideal, 1+np.arange(len(t_ideal))):
    print(i, 'solving at (q, a):', q, a)
    R = [[0, q, 50.0]] # trap center
#     center = [k for k, v in MB_intervals.items() if getOverlap([q-1e-3,q+1e-3], v)]
#     group1 = [max(1, center[0]-1), min(28, center[0]+1)]
#     group2 = [max(1, group1[0]-1), min(28, group1[1]+1)]
# #     group3 = [max(1, group2[0]-1), min(28, group2[-1]+1)]
# #     group4 = [max(1, group3[0]-1), min(28, group3[1]+1)]
#     group0 = [i[0] for i in electrode_grouping if i[0] not in center and i[0] not in group1 and i[0] not in group2]
    
#     notconstrained = [9,10,11,12,29,30,31,32]
#     constrained_voltages = [(i[0]-1, 8) for i in electrode_grouping if i[0] not in notconstrained]
#     constrained_voltages += [(i-1, 0) for i in [29,30,31,32]]
    constrained_voltages = get_constrained_voltages(q, tilt_el={'S25':0, 'S24':0, 'S23':0, 'S22':0})
    print('constrained voltages: ', constrained_voltages)
#     volt_gen.set_electrode_grouping([center]+[[g] for g in group1]+[group2]+[group0])
#     print('electrode grouping: ', volt_gen.electrode_grouping)
    xx, xz, zz = tiltWithPseudo(volt_gen, R, tilt_angle, a, volt_gen.rf_electrodes, debug=False)
    cons = [('x', 0), ('y', 0), ('z', 0), ('yy', a), ('xy', 0), ('yz', 0)]
    nom_v = volt_gen.findControlVoltages(R, cons=[cons], tol=1e-27, fixed_voltages=constrained_voltages, bnds=bounds, normvec=norm, epss=1e-12, independent=True)
    allvoltages.append({electrode_ordering[num-1]: v for [num, v] in volt_gen.ungroup_configuration(nom_v)})
    

1 solving at (q, a): 186.5 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '2.1959410688542928e-08']
 ['y' '0' '-3.18801002307037e-12']
 ['z' '0' '1.4590826376975619e-12']
 ['xy' '0' '1.8878186295510322e-08']
 ['yy' '0.00013093278896406064' '0.00013093278074398135']
 ['yz' '0' '-7.086185479544815e-13']]
Final cost value:  4.845581847692899e-07
Number of iterations:  53 

2 solving at (q, a): 186.5000463510298 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized C

Target, Realized Coeffs:
 [['x' '0' '2.0272368280965066e-08']
 ['y' '0' '3.6412521760807726e-12']
 ['z' '0' '1.403121325724288e-12']
 ['xy' '0' '1.8743347552637146e-08']
 ['yy' '0.00013093278896406064' '0.00013093278106811698']
 ['yz' '0' '6.27112328266486e-12']]
Final cost value:  4.794214849624262e-07
Number of iterations:  51 

14 solving at (q, a): 186.6907655514147 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '2.1143455135588973e-08']
 ['y' '0' '3.708517271484163e-12']
 ['z' '0' '1.4271455111430909e-12']
 ['xy' '0' '1.8754822911312326e-08']
 ['yy' '0.00013093278896406064' '0.0001309327809575401']
 ['yz' '0' '6.409182714173407e-12']]
Final cost value:  4.806410537313037e-07
Number of iterations:  63 

15 

Target, Realized Coeffs:
 [['x' '0' '5.826062400579199e-08']
 ['y' '0' '6.44687612755962e-12']
 ['z' '0' '2.642267264360565e-12']
 ['xy' '0' '1.935271300083825e-08']
 ['yy' '0.00013093278896406064' '0.0001309327770709593']
 ['yz' '0' '1.2629413668834542e-11']]
Final cost value:  5.647454159642894e-07
Number of iterations:  66 

27 solving at (q, a): 189.33554380970682 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '6.390489107049232e-08']
 ['y' '0' '6.521928938747756e-12']
 ['z' '0' '2.8175532653523794e-12']
 ['xy' '0' '1.8741239166014685e-08']
 ['yy' '0.00013093278896406064' '0.0001309327771856618']
 ['yz' '0' '1.3089835720339268e-11']]
Final cost value:  5.671131730530445e-07
Number of iterations:  61 

28 so

Target, Realized Coeffs:
 [['x' '0' '9.874278025538461e-08']
 ['y' '0' '7.81774298075355e-12']
 ['z' '0' '3.748659369029461e-12']
 ['xy' '0' '6.798788339043607e-09']
 ['yy' '0.00013093278896406064' '0.00013093278269651892']
 ['yz' '0' '1.747039818855104e-11']]
Final cost value:  5.221525977176814e-07
Number of iterations:  63 

40 solving at (q, a): 198.06624743024076 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '9.858495735392271e-08']
 ['y' '0' '7.884043244643646e-12']
 ['z' '0' '3.800430998547599e-12']
 ['xy' '0' '8.354440695009553e-09']
 ['yy' '0.00013093278896406064' '0.00013093278320970743']
 ['yz' '0' '1.7474342732894978e-11']]
Final cost value:  5.353484691737673e-07
Number of iterations:  55 

41 sol

Target, Realized Coeffs:
 [['x' '0' '3.9277765785524046e-08']
 ['y' '0' '2.349317788918892e-12']
 ['z' '0' '1.4831522511873918e-12']
 ['xy' '0' '-1.0690177313303353e-08']
 ['yy' '0.00013093278896406064' '0.00013093279002976464']
 ['yz' '0' '4.861821923242242e-12']]
Final cost value:  3.316527094301394e-07
Number of iterations:  47 

53 solving at (q, a): 211.75043344335012 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '2.9571293015219354e-08']
 ['y' '0' '1.841852625278362e-12']
 ['z' '0' '1.0960084294124595e-12']
 ['xy' '0' '-7.953458443295815e-09']
 ['yy' '0.00013093278896406064' '0.0001309327901245688']
 ['yz' '0' '3.721603645916172e-12']]
Final cost value:  2.477851531742912e-07
Number of iterations:  52 



Target, Realized Coeffs:
 [['x' '0' '1.1310838362735206e-07']
 ['y' '0' '8.794536496617433e-12']
 ['z' '0' '6.879780599378105e-12']
 ['xy' '0' '1.7475678881972593e-08']
 ['yy' '0.00013093278896406064' '0.00013093280279781051']
 ['yz' '0' '1.5091515615851293e-11']]
Final cost value:  7.14641533555024e-07
Number of iterations:  52 

66 solving at (q, a): 224.09478481084042 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '1.2392283619220065e-07']
 ['y' '0' '9.443242195150692e-12']
 ['z' '0' '7.464167305071268e-12']
 ['xy' '0' '1.639776175712581e-08']
 ['yy' '0.00013093278896406064' '0.00013093280452120432']
 ['yz' '0' '1.60713638442311e-11']]
Final cost value:  7.429509867536763e-07
Number of iterations:  61 

67 s

Target, Realized Coeffs:
 [['x' '0' '1.7702411251611612e-07']
 ['y' '0' '1.5753659878339765e-11']
 ['z' '0' '9.45895293515131e-12']
 ['xy' '0' '6.712007287961812e-10']
 ['yy' '0.00013093278896406064' '0.00013093282179539096']
 ['yz' '0' '2.1250532705589277e-11']]
Final cost value:  8.852801498291561e-07
Number of iterations:  66 

79 solving at (q, a): 230.24690872324237 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '1.7817232330367415e-07']
 ['y' '0' '1.5811194367665315e-11']
 ['z' '0' '9.548159355179942e-12']
 ['xy' '0' '-1.94238634653596e-09']
 ['yy' '0.00013093278896406064' '0.00013093282186194444']
 ['yz' '0' '2.1262584019312268e-11']]
Final cost value:  8.921846376980549e-07
Number of iterations:  59 

8

Target, Realized Coeffs:
 [['x' '0' '1.8409255904826226e-07']
 ['y' '0' '1.658548829958778e-11']
 ['z' '0' '9.657582025757172e-12']
 ['xy' '0' '-4.4739859894915824e-09']
 ['yy' '0.00013093278896406064' '0.00013093282471047927']
 ['yz' '0' '2.1354682060745027e-11']]
Final cost value:  9.272341637480483e-07
Number of iterations:  50 

92 solving at (q, a): 231.46914771973024 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '1.8414206530776877e-07']
 ['y' '0' '1.660339617529094e-11']
 ['z' '0' '9.6763439275116e-12']
 ['xy' '0' '-4.4810076834668314e-09']
 ['yy' '0.00013093278896406064' '0.00013093282474405666']
 ['yz' '0' '2.135375483749459e-11']]
Final cost value:  9.275010746135449e-07
Number of iterations:  48 

9

## Splitting

In [ ]:
tilt_angle = np.radians(15)
volt_gen = vg
bounds = (-39, 39)
norm = np.full((6,1), 1)
norm[[3, ]] = 1e1
volt_gen.set_electrode_grouping(volt_gen.electrode_grouping)

allvoltages = []
for a, b, i, d in zip(a_ideal, b_ideal, 1+np.arange(len(t_ideal)), d_ideal):
    print(i, 'solving at (a, b):', a, b)
    R = [[0, q0, 52.0]] # trap center
#     notconstrained = [29,30,31,32]
#     notconstrained.extend([k for k, v in MB_intervals.items() if getOverlap([q0-90,q0+90], v)]) # we can tune this parameter
    notconstrained = [9,10,11,12,13, 29,30,31,32]
    constrained_voltages = [(i[0]-1, 8) for i in electrode_grouping if i[0] not in notconstrained]
    constrained_voltages += [(i-1, 0) for i in [29,30,31,32]]
    print('constrained voltages: ', constrained_voltages)
#     xx, xz, zz = tiltWithPseudo(volt_gen, R, tilt_angle, a, volt_gen.rf_electrodes, debug=False)
    cons = [[('x', 0), ('y', 0), ('z', 0), ('yy', a), ('yyy', 0), ('yyyy', b)]]
    nom_v = volt_gen.findControlVoltages(R, cons=cons, tol=1e-27, fixed_voltages=constrained_voltages, bnds=bounds, normvec=norm, epss=1e-12, independent=False)
    allvoltages.append({electrode_ordering[num-1]: v for [num, v] in volt_gen.ungroup_configuration(nom_v)})
    

In [12]:
allvoltages = [[]]*len(a_ideal)
print(allvoltages)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [17]:
#### Taylor fit splitting
# a, b = (-5.8369100774619e-06, 1.0175595418061765e-08) # double well configuration (for testing)
q0 = 186.5

volt_gen = vg
bounds = (-39, 39)
norm = np.full((4,1), 1)
norm[[3]] = 1e2
volt_gen.set_electrode_grouping(electrode_grouping)

indices = [8, 11, 12, 13, 14, 16, 17, 21, 22, 29, 30, 31, 33]
# allvoltages = []
# for a, b, d in zip(a_ideal[:], b_ideal[:], d_ideal[:]):
for i in indices:
    a,b,d = a_ideal[i], b_ideal[i], d_ideal[i]
    print('solving at (a, b):', a, b)
    R = [[0, q0, 51.0]] # trap center
    notconstrained = [[9],[10],[11],[12],[13],[29],[30],[31],[32]] ## act
    constrained_voltages = [(i, 8) for i, g in enumerate(electrode_grouping) if g not in notconstrained]
    constrained_voltages += [(i, 0) for i, g in enumerate(electrode_grouping) if g in [[29],[30],[31],[32]]]
    print('constrained voltages: ', constrained_voltages)
    cons = [[('y',0),('yy',a),('yyy',0),('yyyy',b)]]
    nom_v = findControlVoltages2(R, cons=cons, tol=1e-15, fixed_voltages=constrained_voltages, bnds=bounds, normvec=norm, epss=1e-8, independent=True, di=float(d))
#     allvoltages.append({electrode_ordering[num-1]: v for [num, v] in volt_gen.ungroup_configuration(nom_v)})
    allvoltages[i] = {electrode_ordering[num-1]: v for [num, v] in volt_gen.ungroup_configuration(nom_v)}
    

solving at (a, b): 8.146813407449812e-06 8.444867942842854e-08
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]


/Users/lukeqi/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Optimization failed after 2001 iterations.
Target, Realized Coeffs:
 [['y' '0' '-2.940707396089795e-11']
 ['yy' '8.146813407449812e-06' '8.146813876830059e-06']
 ['yyy' '0' '5.219867121042481e-12']
 ['yyyy' '8.444867942842854e-08' '8.444869046839116e-08']]
Final cost value:  1.6279625615376644e-07
Number of iterations:  2001 

solving at (a, b): 4.397925862633755e-06 1.3941140097033776e-07
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['y' '0' '7.283396722381078e-11']
 ['yy' '4.397925862633755e-06' '4.397914617588579e-06']
 ['yyy' '0' '-1.5868126643201836e-12']
 ['yyyy' '1.3941140097033776e-07' '1.394113995040492e-07']]
Final cost value:  4.1426558581929603e-08
Number of iterations:  326 

solving at (a, b): 3.1064557714707004e-06 1.4977946964506981e

In [16]:
# splittingvoltages = [[]]*100
# splittingvoltages[7:9] = allvoltages
splittingvoltages = allvoltages
print(splittingvoltages)

[{'S21': 8.0, 'N1': 8.0, 'S20': 8.0, 'N2': 8.0, 'S19': 8.0, 'N3': 8.0, 'S14': 8.0, 'N4': 8.0, 'S13': -1.2764874378599698, 'N8': 0.6539949261158626, 'S12': 0.540321602024275, 'N9': 0.6475366980834651, 'S11': -1.2957754238746089, 'N10': 8.0, 'S10': 8.0, 'N11': 8.0, 'S9': 8.0, 'N12': 8.0, 'S8': 8.0, 'N13': 8.0, 'S4': 8.0, 'N14': 8.0, 'S3': 8.0, 'N19': 8.0, 'S2': 8.0, 'N20': 8.0, 'S1': 8.0, 'N21': 8.0, 'S25': 0.0, 'S24': 0.0, 'S23': 0.0, 'S22': 0.0}, {'S21': 8.0, 'N1': 8.0, 'S20': 8.0, 'N2': 8.0, 'S19': 8.0, 'N3': 8.0, 'S14': 8.0, 'N4': 8.0, 'S13': -1.2104178934904668, 'N8': 0.6457680290752766, 'S12': 0.5556482008971826, 'N9': 0.6393171426634178, 'S11': -1.2297210110052452, 'N10': 8.0, 'S10': 8.0, 'N11': 8.0, 'S9': 8.0, 'N12': 8.0, 'S8': 8.0, 'N13': 8.0, 'S4': 8.0, 'N14': 8.0, 'S3': 8.0, 'N19': 8.0, 'S2': 8.0, 'N20': 8.0, 'S1': 8.0, 'N21': 8.0, 'S25': 0.0, 'S24': 0.0, 'S23': 0.0, 'S22': 0.0}, {'S21': 8.0, 'N1': 8.0, 'S20': 8.0, 'N2': 8.0, 'S19': 8.0, 'N3': 8.0, 'S14': 8.0, 'N4': 8.0, 'S13'

In [ ]:
for i, a in enumerate(a_ideal):
    print(i, a)

## Just Tilt

In [ ]:
t_ideal = np.linspace(0, 10, 30)
a_ideal = np.zeros(len(t_ideal))
q_ideal = np.linspace(-746, 746, len(t_ideal))

tilt_angle = np.radians(10) # specify tilt [rad]
volt_gen = vg
bounds = (-39.5, 39.5)
norm = np.full((5, 1), 1) # normalization factor to pull out certain coefficients
volt_gen.set_electrode_grouping(electrode_grouping) # sets previous voltages to None

allvoltages = []
for q, a, i in zip(q_ideal, a_ideal, 1+np.arange(len(t_ideal))):
    print(i, 'solving at (q, a):', q, a)
    R = [[0, q, 50.0]] # trap center
    
    notconstrained = [29,30,31,32]
    constrained_voltages = [(i[0]-1, 0) for i in electrode_grouping if i[0] not in notconstrained]
    print('constrained voltages: ', constrained_voltages)
#     xx, xz, zz = tiltWithPseudo(volt_gen, R, tilt_angle, a, volt_gen.rf_electrodes, debug=False)
    cons = [('x',0), ('z',0), ('xx', -0.0001883908540906727), ('xz', 0.000148582933577121), ('zz', 0.00013725619957816228)]
    nom_v = volt_gen.findControlVoltages(R, cons=[cons], tol=1e-27, fixed_voltages=constrained_voltages, bnds=bounds, normvec=norm, epss=1e-12, independent=True)
    allvoltages.append({electrode_ordering[num-1]: v for [num, v] in volt_gen.ungroup_configuration(nom_v)})
    

## Just Shim

In [ ]:
tilt_angle = np.radians(10) # specify tilt [rad]
volt_gen = vg
bounds = (-39.5, 39.5)
norm = np.full((6, 1), 1) # normalization factor to pull out certain coefficients
volt_gen.set_electrode_grouping(electrode_grouping) # sets previous voltages to None

allvoltages = []
R = [[0, 186.5, 50.0]] # trap center

notconstrained = [9,10,11,12,13]
constrained_voltages = [(i[0]-1, 0) for i in electrode_grouping if i[0] not in notconstrained]
print('constrained voltages: ', constrained_voltages)
#     xx, xz, zz = tiltWithPseudo(volt_gen, R, tilt_angle, a, volt_gen.rf_electrodes, debug=False)
for z in [1e-4, 2e-4, 3e-4]:
    cons = [('x',0), ('y',0), ('z',z), ('yy',0), ('zz',0), ('xx',0)]
    nom_v = volt_gen.findControlVoltages(R, cons=[cons], tol=1e-27, fixed_voltages=constrained_voltages, bnds=bounds, normvec=norm, epss=1e-12, independent=True)
    allvoltages.append({electrode_ordering[num-1]: v for [num, v] in volt_gen.ungroup_configuration(nom_v)})

print(allvoltages)
    

## Bucket

In [11]:
import itertools

def ffity(y, a, b, c, d, const):
    return a*y + b*y**2 + c*y**3 + d*y**4 + const
    
def get_realizedcoeff(voltages, fixed_voltage_indices, fixed_voltage_values, vg, di=0):
    voltages = np.insert(voltages, fixed_voltage_indices, fixed_voltage_values)
    voltages = [{electrode_ordering[num-1]: v for [num, v] in vg.ungroup_configuration(voltages)}]

    d = defaultdict(list)
    for f in voltages:
        [d[k].append(v) for k, v in f.items()]
    bigarray = np.array([d[k] for k in electrode_ordering]).T
    el_config = []
    for row in bigarray:
        el_configi = [[[i+1], v] for i, v in enumerate(row)]
        el_configi.extend([[[el], 1] for el in vg.rf_electrodes])
        el_config.append(el_configi)
    y_span_um = (-30, 30)
    n_y_points = int(1e-6*(y_span_um[1]-y_span_um[0])/1000e-9)+1 # set dy (space resolution)
    y_points, dy = np.linspace(y_span_um[0], y_span_um[1], n_y_points, retstep=True)
    y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
    y_pot = np.array([vg.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config])
    xaxis = y_points
    yaxis = y_pot
    
    popt, pcov = curve_fit(ffity, xaxis, yaxis[0], p0=[0, 0, 0, 0, 0], maxfev=10000)
    realized = popt[:-1]
    
    zshim = 0
    R1 = [[0.0, 186.5+di, 51]]
    R2 = [[0.0, 186.5-di, 51]]
    z_span_um = (-9, 9)
    n_z_points = int(1e-6*(z_span_um[1]-z_span_um[0])/1000e-9)+1
    z_points, dz = np.linspace(z_span_um[0], z_span_um[1], n_z_points, retstep=True)
    for r in [R1, R2]:
        z_test_points = np.array([np.zeros(n_z_points), np.zeros(n_z_points), z_points]).T + r
        z_pot = np.array([volt_gen.compute_potential(z_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config])
        zshim += abs(np.gradient(z_pot[0])[int(-z_span_um[0]/dz)])
    
    return realized, zshim
    
def findControlVoltages2(
            R, cons=d_cons, bnds=(-np.inf, np.inf),
            tol=1E-15, pseudoCorrection=False, dv_weight=0, max_dv=1e6,
            independent=True, fixed_voltages = [], normvec=None, epss=1e-4, di=0
    ):
        """
        Given an expansion points and a set of target coefficients, solves the least-
          squares problem for Cv=y for v, subject to additional constraints to keep
          voltages within acceptable ranges and keep waveforms smooth during transport.

        Parameters
        ----------
        R : array_like, reshapable to (3,)
            expansion point
        cons : ???
            set of constraints on the coefficients of the Taylor series
        bnds : tuple
            maximum range of voltages
        tol : ???
            ???
        pseudoCorrection : boolean
            not implemented; would be a flag to include the pseudopotential in the fit
        dv_weight : float
            weighting factor applied to the Euclidian distance between voltage sets
        max_dv : float
            ???
        independent : boolean
            when true, do not use the previous voltage solution in the constraints
        independent : list of tuples
            each tuple contains the (index, voltage) for a constrained electrode

        Returns
        -------
        ret_val.x : ndarray
            Voltage arrray found using least squares optimization.
        """

        fixed_voltages.sort(key=lambda cond: cond[0])
        fixed_voltage_indices = [cond[0] for cond in fixed_voltages]
        fixed_voltage_values = [cond[1] for cond in fixed_voltages]

        for i in range(len(fixed_voltage_indices)):
            fixed_voltage_indices[i] -= i

        # JUPDATE: Is there a safer way to handle cases like when you switch which electrodes are constrained between fits?
        if (independent
                or vg.prev_voltages is None
                or len(vg.prev_voltages) != len(vg.electrode_grouping, ) - len(fixed_voltages)):
            vg.prev_voltages = np.zeros((len(vg.electrode_grouping, ) - len(fixed_voltages)))

        # Ensures that BOTH the default coefficients and the supplied coefficients
        #   are used in the generating of the C matrix; also sorts the coefficients
        #   so they're in the correct, ascending order
        tcons = []
        for conset in cons:
            # NOTE: Some older notebooks could throw an error here; make sure that
            #  vg.d_cons is a list of tuples and not a list of lists
            tcons += [sorted(dict(d_cons + conset).items(),
                key=lambda x: int(vg.encode_string(x[0])))]
        C = vg.generateCMatrix(R, cons=tcons)

        # not implemented; intended for doing fits including the pseudopotential
        # pseudoFit = np.zeros(1)

        # Converts the target coefficient constraints in cons, which can include
        #   multiple points, into the vector y
        target_coeffs = np.zeros(1)
        order_coeffs = np.zeros(1)

        for i in range(np.shape(R)[0]):
            target_cons = {}
            # Populate the vector with the constraints
            for (k, v) in tcons[i]:
                target_cons[k] = v

            # Masks the constraint vector to only hold the desired coefficients,
            #   instead of all of the coefficients
            mask_vec = np.zeros((len(vg.fit_functions.keys()), 1), dtype=bool)
            target_vec = np.zeros((len(vg.fit_functions.keys()), 1))
            order_vec = np.zeros((len(vg.fit_functions.keys()), 1), dtype=int)

            for k in target_cons.keys():
                # canonical integer associated with the constraint
                # (e.g. 'xx' might be the 4th constraint)
                constraint_index = vg.function_map[k]
                target_vec[constraint_index] = target_cons[k]
                mask_vec[constraint_index] = True
                order_vec[constraint_index] = 0 if k == 'const' else len(k)

            target_vec = np.reshape(target_vec[mask_vec], (-1, 1))
            order_vec = np.reshape(order_vec[mask_vec], (-1, 1))
            target_coeffs = np.vstack((target_coeffs, target_vec))
            order_coeffs = np.vstack((order_coeffs, order_vec))

        target_coeffs = target_coeffs[1::]

        # we normalize coefficients based on the order of their monomial
        # quadratic gets multiplied by d^2, quartic by d^4, etc.
        # where d = vg.trap_length_scale is a characteristic length scale
        # of the trap
        # intuition: the badness of an error should correspond to errors in
        # the generated potential; also, this normalization makes our code
        # dimensionally correct.
        normalization_vec = np.power(vg.trap_length_scale, order_coeffs[1::])

        if normvec is not None:
            normalization_vec *= normvec
            
        def cost(voltages):
            dv = la.norm(vg.prev_voltages.reshape(-1)-voltages.reshape(-1))
            total_cost = dv*dv_weight
            
            realized, zshim = get_realizedcoeff(voltages, fixed_voltage_indices, fixed_voltage_values, vg, di)
            sq_diff = la.norm(
                (realized.reshape(-1)-target_coeffs.reshape(-1))*normalization_vec.reshape(-1)
            )
            total_cost = total_cost + sq_diff
            
#             voltage_norm = la.norm(voltages)
#             total_cost += voltage_norm*1e-6
            total_cost += zshim*1e-2

            return total_cost

        delta_v = lambda x: max_dv - np.max(np.abs(x-vg.prev_voltages))
        cdict = ({'type': 'ineq', 'fun': delta_v})
        boundss = np.repeat([bnds], len(vg.electrode_grouping)-len(fixed_voltages), axis=0)
        ret_val = sciopt.minimize(
            cost,
            method='SLSQP',
            x0=vg.prev_voltages,
            #x0=np.zeros((len(vg.electrode_grouping, ))),
            tol=tol,
            constraints=cdict,
            options={'maxiter':2000, 'eps':epss},
            bounds=boundss,
        )

        if not ret_val.success:
            print("Optimization failed after %i iterations."%(ret_val.nit))

        output_voltages = np.insert(ret_val.x, fixed_voltage_indices, fixed_voltage_values)
        realized_coeffs, _ = get_realizedcoeff(output_voltages, [], [], vg)
        print('Target, Realized Coeffs:\n', np.concatenate([np.array(list(itertools.chain.from_iterable(tcons))),
                                                            realized_coeffs.reshape(-1,1)], axis=1))
        print('Final cost value: ', ret_val.fun)
        print('Number of iterations: ', ret_val.nit, '\n')
        vg.prev_voltages = ret_val.x
        return output_voltages

In [ ]:
#### DOES NOT WORK (onion grouping)
# a, b = (2.5877307916717177e-06, 2.3461213568834696e-08) # values for 5um spacing (from the optimization in sec 7.3)
# a, b = (3.2327258901248085e-07, 7.335607430242796e-10) # values for 10um spacing
a, b = (-5.8369100774619e-06, 1.0175595418061765e-08) # double well configuration (for testing)
q0 = 186.5

volt_gen = vg
bounds = (-39, 39)
norm = np.full((4,1), 1)
norm[[3]] = 1e2
newgrouping = electrode_grouping[:7]+[[8,14]]+[[9,13]]+[[10,12]]+[[11]]+electrode_grouping[14:]
volt_gen.set_electrode_grouping(newgrouping)

allvoltages = []
print('solving at (a, b):', a, b)
R = [[0, q0, 51.0]] # trap center
notconstrained = [[8,14],[9,13],[10,12],[11],[29],[30],[31],[32]] ## act
constrained_voltages = [(i, 8) for i, g in enumerate(newgrouping) if g not in notconstrained]
constrained_voltages += [(i, 0) for i, g in enumerate(newgrouping) if g in [[29],[30],[31],[32]]]
print('constrained voltages: ', constrained_voltages)
cons = [[('y',0),('yy',a),('yyy',0),('yyyy',b)]]
nom_v = findControlVoltages2(R, cons=cons, tol=1e-27, fixed_voltages=constrained_voltages, bnds=bounds, normvec=norm, epss=1e-6, independent=True)
allvoltages.append({electrode_ordering[num-1]: v for [num, v] in volt_gen.ungroup_configuration(nom_v)})
    

In [ ]:
# a, b = (2.5877307916717177e-06, 2.3461213568834696e-08) # values for 5um spacing (from the optimization in sec 7.3)
# a, b = (3.2327258901248085e-07, 7.335607430242796e-10) # values for 10um spacing
a, b = (-5.8369100774619e-06, 1.0175595418061765e-08) # double well configuration (for testing)
q0 = 186.5

volt_gen = vg
bounds = (-39, 39)
norm = np.full((4,1), 1)
norm[[3]] = 1e2
volt_gen.set_electrode_grouping(electrode_grouping)

allvoltages = []
print('solving at (a, b):', a, b)
R = [[0, q0, 51.0]] # trap center
notconstrained = [[9],[10],[11],[12],[13],[29],[30],[31],[32]] ## act
constrained_voltages = [(i, 8) for i, g in enumerate(electrode_grouping) if g not in notconstrained]
constrained_voltages += [(i, 0) for i, g in enumerate(electrode_grouping) if g in [[29],[30],[31],[32]]]
print('constrained voltages: ', constrained_voltages)
cons = [[('y',0),('yy',a),('yyy',0),('yyyy',b)]]
nom_v = findControlVoltages2(R, cons=cons, tol=1e-15, fixed_voltages=constrained_voltages, bnds=bounds, normvec=norm, epss=1e-9, independent=True)
allvoltages.append({electrode_ordering[num-1]: v for [num, v] in volt_gen.ungroup_configuration(nom_v)})
    

In [ ]:
## view voltages in pretty format
print('\t'.join('{}={:.3f}V'.format(k, v) for k, v in allvoltages[0].items()))

# View Ideal Voltages

**Load allvoltages from npy files** (not always necessary)

In [ ]:
basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filepath = 'MB_transport_5us_c2c_4MHz/'
filenames = sorted(os.listdir(''.join([basepath, filepath])), key = lambda x: float(x.split('_')[-1][:-4]))
allvoltages = [np.load(''.join([basepath, filepath]) + fname, allow_pickle = True).tolist() for fname in filenames[:]]

In [ ]:
basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filepath = 'MB_1200kHz_split_186um_notilt.npy'
# filepath = 'MB_split_1200kHz_grandcanyon.npy'
# filepath = 'MB_11-10transport_notilt.npy'
# filepath = 'MB_split_1000kHz_zerotilt_z=51um.npy'
allvoltages = np.load(''.join([basepath, filepath]), allow_pickle = True).tolist()

d = defaultdict(list)
for f in allvoltages:
    [d[k].append(v) for k, v in f.items()]
    

In [52]:
basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
# filepath = 'MB_11-10transport_notilt_2steps.npy'
# filepath = 'MB_fasttransport_4000kHz_better.npy'
# filepath = 'MB_fasttransport_4MHz_dt=392ns_off.npy'
filepath = 'MB_fasttransport_4000kHz_dt=800ns.npy'
allvoltages = np.load(''.join([basepath, filepath]), allow_pickle = True).tolist()

dt = 800e-9

v_DAC = defaultdict(list)
for f in allvoltages:
    [v_DAC[k].append(v) for k, v in f.items()]
    
T0 = dt*len(v_DAC['S13'])
print(v_DAC)

defaultdict(<class 'list'>, {'S21': [8, 8, 8, 8, 8, 8, 8], 'N1': [8, 8, 8, 8, 8, 8, 8], 'S20': [8, 8, 8, 8, 8, 8, 8], 'N2': [8, 8, 8, 8, 8, 8, 8], 'S19': [8, 8, 8, 8, 8, 8, 8], 'N3': [8, 8, 8, 8, 8, 8, 8], 'S14': [8, 8, 8, 8, 8, 8, 8], 'N4': [8, 8, 8, 8, 8, 8, 8], 'S13': [8.070269881540078, -14.751603604392722, 16.51682941303978, -3.0609114741011783, 10.612934856150583, -4.624559452564909, 1.1086299734165868], 'N8': [1.1017828519430135, 0.061824159324150675, -2.9689481288352364, 7.457175685502516, -4.1853470054152515, 1.8648865971173985, 0.6549658001952307], 'S12': [0.6660767060801299, -2.805258582487428, 10.543547849835305, -11.468219960180125, 9.701728351282295, -1.5135446426538763, 1.115832086660291], 'N9': [1.1099136371439717, 15.290892306867958, 14.321247199780965, 1.863794994610247, 0.841053524512099, 12.869169201973403, 7.946760109488787], 'S11': [8, 8, 8, 8, 8, 8, 8], 'N10': [8, 8, 8, 8, 8, 8, 8], 'S10': [8, 8, 8, 8, 8, 8, 8], 'N11': [8, 8, 8, 8, 8, 8, 8], 'S9': [8, 8, 8, 8, 8,

In [ ]:
v_DAC['S13'] = [  8.07026988,   8.07026988,   8.07026988,   8.07026988,
       -11.57455123, -11.57455123, -11.57455123, -11.57455123,
        19.65472765,  19.65472765,  19.65472765,  19.65472765,
        -5.44338066,  -5.44338066,  -5.44338066,  -5.44338066,
        -2.4815944 ,  -2.4815944 ,  -2.4815944 ,  -2.4815944 ,
         40,   5.3861852491460365,   5.3861852491460365,   5.3861852491460365,
         1.10862997,   1.10862997,   1.10862997,   1.10862997]

In [21]:
d = defaultdict(list)
for f in allvoltages:
    [d[k].append(v) for k, v in f.items()]

# T0 = 100e-6
dt = 1e-6 # sample rate of DAC
N0 = int(T0/dt)
buffer1 = 1
buffer2 = 1
Ntot = buffer1+N0+buffer2

v_ideal = {} # voltages from backwards ITVG
v_DAC = {} # sampled voltages for DAC commands
plt.figure('voltages')
for k, v in d.items():
    mark = '.'
    if k[0] == 'S':
        mark = 'x'

    vevents = np.concatenate(([v[0]], v, [v[-1]]))
    tevents = np.concatenate(([-0.1e-6], dt*np.linspace(buffer1, N0+buffer1, len(v)), [Ntot*dt+0.1e-6]))

#     tmiddle = dt*np.linspace(buffer1, N0+buffer1, 500)
#     vmiddle = InterpolatedUnivariateSpline(dt*np.linspace(buffer1, N0+buffer1, len(v)), v, k=3).__call__(tmiddle)
#     vevents = np.concatenate(([v[0]], vmiddle, [v[-1]]))
#     tevents = np.concatenate(([-0.1e-6], tmiddle, [Ntot*dt+0.1e-6]))
#     vevents = savgol_filter(vevents, 51, 1)
#     mark=''
    if k not in []: #['S22', 'S23', 'S24', 'S25']:
        plt.plot([t*1e6 for t in tevents], vevents, marker=mark, label=k)
#         plt.plot(np.concatenate([[q_ideal[0]],q_ideal,[q_ideal[-1]]]), vevents, marker=mark, label=k)
    v_ideal[k] = interp1d(np.append(tevents, 1), np.append(vevents, vevents[-1]))
    v_DAC[k] = v_ideal[k].__call__([(n+0)*dt for n in range(Ntot)])
plt.legend(loc='best', prop={'size': 12})
plt.ylabel('Electrode Voltage (V)')
plt.xlabel('Time (us)')
# plt.xlabel('Position (um)')
endsim = dt*len(v_DAC['N9'])


**Save Voltages**

In [19]:
#### save allvoltages
basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
# filepath = 'MB_MBZ2-MBZ3_2500kHz.npy'
filepath = 'MB_split_max.npy'

np.save(os.path.join(basepath, filepath), allvoltages)

In [ ]:
#### save each voltage dict separately (allvoltages is a list of dicts)
basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filepath = 'SLT_split_300us_STA/'

for i, v in enumerate(allvoltages):
    savefilepath = os.path.join(basepath, filepath, "NominalVolts_{}.npy".format(i+1))
    np.save(savefilepath, v)
    

In [29]:
#### save v(t) for each electrode separately
basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filepath = 'predistortion4/'

t_ideal = np.linspace(0, Ntot*dt, 100)
for k, v in v_ideal.items():
    if k in ['N8', 'N9', 'S12', 'S13']:
        plt.plot(t_ideal, v.__call__(t_ideal))
        savefilepath = os.path.join(basepath, filepath, "{}_ideal_withbuffers.npy".format(k))
        np.save(savefilepath, v.__call__(t_ideal))
        

In [ ]:
#### saves voltages as a list of dicts
basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filepath = 'MB_twostagesplit_1000kHz_zerotilt_z=52um.npy'

newdat = []
for i in range(len(v_DAC['N10'])):
    newdat.append({k: v[i] for k, v in v_DAC.items() if k not in ['S22', 'S23', 'S24', 'S25']})
newdat = np.array(newdat)

# tilt_el={'S25': 12.930878042522904, 'S24': -24.036409224366135, 'S23': -39.5, 'S22': 27.90731149920946}
tilt_el={'S25': 0, 'S24': 0, 'S23': 0, 'S22': 0}

for dic in newdat:
    for k in ['S22', 'S23', 'S24', 'S25']:
        dic[k] = tilt_el[k]
print(newdat)
np.save(os.path.join(basepath, filepath), newdat) 


# Simulate Trap Effects
**Potential sources of noise include: DAC slew rate, calibration and gain offset, crosstalk, discrete inputs, filtering, segmentation of electrodes**

Important to characterize between physical and numerical effects

In [ ]:
#### Visualizing Jules's voltages ####

basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filepath = 'SLT_split_300us_Kaufmann.npy'
dat = np.load(os.path.join(basepath, filepath), allow_pickle=True).tolist()

zotino_ordering = [ # Standard Lincoln trap
        "N6", "N7", "N8", "N9", "N10", "N11", "N12", "N13", "N14",
        "S6", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14",
        "S24", "S25"]

# v_DAC = {}
# for i, drow in enumerate(np.array(dat).T):
#     v_DAC[zotino_ordering[i]] = np.array(drow)*4
    
v_DAC = {k:v[::3] for k, v in dat.items()}

dt = 2e-6
T0 = dt*len(v_DAC['S6'])
for k, v in v_DAC.items():
    plt.plot(1e6*np.linspace(0, int(T0/dt)-1, int(T0/dt))*dt, list(v), marker='.')
    
print(len(v_DAC['S6']))


## Simulate Electrode Response

In [56]:
def set_vinput(volt_array, dt, st, sr):
    ''' writes PWL input to 'input.txt' '''    
    N = len(volt_array)
    varray = [v/16+2.5 for v in volt_array] ## fastino
#     varray = volt_array ## zotino
    
    tevents = [0]
    vevents = [varray[0]]
    for i in range(1,N):
        ti = dt*i
        vi = vevents[-1]
        tevents.append(ti)
        vevents.append(vi)
        
        tdelay = abs(varray[i] - vi)/sr
#         tdelay = 1e-12
        if tdelay > dt:
            print('WARNING: SLEW RATE LONGER THAN TIME STEP')
            tevents.append(ti + dt)
            vevents.append(vi + (varray[i] - vi)*dt/tdelay)
        else:
            tevents.append(ti + tdelay)
            vevents.append(varray[i])
    tevents.append(N*dt)
    vevents.append(varray[-1])
    
    with open(circuit_directory_path + 'input.txt', 'w+') as f:
        [f.write('\t'.join((str(t), str(v))) + '\n') for t, v in zip(tevents, vevents)]

    return tevents, vevents

def run_spice_sim(endsim, st, attempts=1):
    instruction = '.TRAN' + ' '.join((str(st), str(endsim)))
    LTC.netlist.insert(-2, instruction)
    LTC.write_netlist()
    rawfile, logfile = LTC.run()
    LTC.reset_netlist()

    if rawfile:
        for i in range(attempts):
            try:
                LTR = LTSpiceRawRead(rawfile)
                break
            except UnicodeDecodeError:
                LTC.netlist.insert(-2, instruction)
                LTC.write_netlist()
                rawfile, logfile = LTC.run()
                LTC.reset_netlist()
                LTR = LTSpiceRawRead(rawfile)
        os.remove(logfile)
    else:
        print('SIMULATION FAILED')
        with open(logfile, 'r') as f:
            [print(line) for line in f.readlines()]
        os.remove(logfile)
        return None
    return LTR

circuit_name = 'FastinoAmplifier_V1.0.cir' ## Zotino vs FastinoAmplifier vs minimalfiltering

circuit_directory_path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/lib_circuits/'
LTC = LTCommander(circuit_directory_path + circuit_name)
endsim = T0
st = max(dt/20, 1e-9) ## maximum time step of the SPICE transient sim. no lower than 100ns
slew_rate = 17/1e-6 ## slew rate in V/s (~17V/us)
gain_err = 0.01 ## 1%
calibration_err = 0.1 ## +- 100mV

print(dt/st, 'sim steps per dt')

20.0 sim steps per dt


- DAC discrete commands, slew rate, and filtering are naturally taken care of.
- we can artificially add calibration + gain offsets, as well as crosstalk...
- anything else??? John: johnson noise? electric field noise?

In [57]:
plotting = True
addnoise = False
endbuffer = 30e-6/dt

v_exp = {}
for k, v in v_DAC.items():
    if k in ['N8','N9','S12','S13']:
        endsim = dt*(len(v) + endbuffer)
        tevents, vevents = set_vinput(v[::], dt, st, slew_rate)

        LTR = run_spice_sim(endsim, st, 3)

        analysis_output = LTR.get_trace('V(ion)').get_wave(0)
        t_spice = LTR.get_trace('time').get_time_axis(0)
        print('\t'.join((k, str(v), str(t_spice[-1]))), '\n')

        if addnoise:
            v_exp[k] = interp1d(t_spice, np.array(analysis_output)*(1+random.choice([-gain_err, gain_err]))
                             + random.choice([-calibration_err, calibration_err]))
        else:
            v_exp[k] = interp1d(np.concatenate([t_spice, [1]]), np.concatenate([analysis_output, [analysis_output[-1]]]))

        if plotting:
            plt.figure(k, (19, 9))
            plt.title('Electrode %s' % (k, ))
            plt.xlabel('Time [us]')
            plt.ylabel('Voltage [V]')
            plt.grid()

            t_view = np.linspace(0, t_spice[-1], 1000)
            plt.plot([t*1e6 for t in tevents], [16*(v-2.5) for v in vevents], label='input') ## fastino
    #             plt.plot([t*1e6 for t in tevents], vevents, label='input') ## zotino
            plt.plot(t_spice*1e6, analysis_output, label='output')
            plt.plot(t_view*1e6, v_ideal[k].__call__(t_view), 'k--', label='ideal')
            plt.legend(loc='best')

            plt.tight_layout()
plt.show()

Sun Jun 27 22:56:42 2021 : Starting simulation 1
Sun Jun 27 22:56:42 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
S13	[8.070269881540078, -14.751603604392722, 16.51682941303978, -3.0609114741011783, 10.612934856150583, -4.624559452564909, 1.1086299734165868]	3.56e-05 

Sun Jun 27 22:56:42 2021 : Starting simulation 2
Sun Jun 27 22:56:44 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
N8	[1.1017828519430135, 0.061824159324150675, -2.9689481288352364, 7.457175685502516, -4.1853470054152515, 1.8648865971173985, 0.6549658001952307]	3.561577148437468e-05 

Sun Jun 27 22:56:44 2021 : Starting simulation 3
Sun Jun 27 22:56:46 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
S12	[0.6660767060801299, -2.805258582487428, 10.543547849835305, -11.468219960180125, 9.701728351282295, -1.5135446426538763, 1.115832086660291]	3.560522994902625e-05 

Sun Jun 27 22:56:46 2021 : Starting simulation 4
Sun Jun 27 22:56:48 2021: Simulation Successful

In [ ]:
#### Debriefing from Feb 25 hackathon
t_dat = np.load("hackathon_feb25/5us_linear_times_array_list.npy", allow_pickle=True)
v_dat = np.load("hackathon_feb25/5us_linear_volts_array_list.npy", allow_pickle=True)

for t, v in zip(t_dat, v_dat):
    plt.plot(t*1e6, v)
    
for k in ['S10', 'N10']:
    plt.plot(t_dat[0]*1e6, v_exp[k].__call__(np.maximum(0, t_dat[0]-80e-6)), 'k')


In [ ]:
#### Debriefing from Mar 1 hackathon
t_dat = np.load("03.01.21/scope_captures/6us_linear_10um_step_2400kHz_wider_times_array_list.npy", allow_pickle=True)
v_dat = np.load("03.01.21/scope_captures/6us_linear_10um_step_2400kHz_wider_volts_array_list.npy", allow_pickle=True)

for t, v in zip(t_dat, v_dat):
    plt.plot(t*1e6, v)
    
# for k in ['S10', 'N10']:
#     plt.plot(t_dat[0]*1e6, v_exp[k].__call__(np.maximum(0, t_dat[0]-80e-6)), 'k')


In [ ]:
t_all = []
v_exp = {}
for k, v in v_DAC.items():
    tevents, vevents = set_vinput(v, dt, st, slew_rate)
#     v_exp[k] = interp1d(np.concatenate([tevents, [1]]), np.concatenate([[16*(v-2.5) for v in vevents], [16*(vevents[-1]-2.5)]]))
    v_exp[k] = interp1d(np.concatenate([tevents, [1]]), np.concatenate([vevents, vevents[-1:]]))
    t_all = np.concatenate([t_all, tevents])

t_all = np.unique(sorted(t_all))
print(len(t_all))
print(t_all)


In [ ]:
for k, v in v_ideal.items():
    t_sample = np.linspace(0, tevents[-1], 5000)
    v_exp[k] = interp1d(np.concatenate([t_sample, [1]]), np.concatenate([v.__call__(t_sample), [v.__call__(t_sample[-1])]]))

In [ ]:
print(len(v_ideal.keys()))

## Calculate Potential

**Configure voltage output**

**DON'T FORGET TO CHOOSE THE PROPER VOLTAGES**

In [ ]:
zshim = {'S21': 0.0, 'N1': 0.0, 'S20': 0.0, 'N2': 0.0, 'S19': 0.0, 'N3': 0.0, 'S14': 0.0, 'N4': 0.0, 'S13': 0.4484057326527151, 'N8': -0.13554507789550316, 'S12': 0.06742756883327992, 'N9': -0.13561294997392126, 'S11': 0.44870002610895593, 'N10': 0.0, 'S10': 0.0, 'N11': 0.0, 'S9': 0.0, 'N12': 0.0, 'S8': 0.0, 'N13': 0.0, 'S4': 0.0, 'N14': 0.0, 'S3': 0.0, 'N19': 0.0, 'S2': 0.0, 'N20': 0.0, 'S1': 0.0, 'N21': 0.0, 'S25': 0.0, 'S24': 0.0, 'S23': 0.0, 'S22': 0.0}
new_d = {}
n = 10
for k, v in zshim.items():
    d[k] = list(np.array(d[k]) + n*v)

In [88]:
endITVG = endsim - 0e-6 # we can't find the potential for voltages that don't exist! hence the minus 0.1us
volt_gen = vg
N_final = int(endITVG/100e-9)+1 # set dt (time resolution)
N_final = len(d['S21'])
t_exp, dx = np.linspace(0, (endITVG), N_final, retstep=True)
# t_exp = np.unique(sorted(np.concatenate((t_all, t_exp))))
print(len(t_exp), 'calls per iteration \t', endITVG, 'us')

## DON'T FORGET TO CHOOSE PROPER VOLTAGES

# bigarray = np.array([v_exp[k].__call__(t_exp) for k in electrode_ordering]).T
# bigarray = np.array([v_DAC[k] for k in electrode_ordering]).T
bigarray = np.array([d[k] for k in electrode_ordering]).T
el_config = []
for row in bigarray:
    el_configi = [[[i+1], v] for i, v in enumerate(row)]
    el_configi.extend([[[el], 1] for el in volt_gen.rf_electrodes])
    el_config.append(el_configi)
    

100 calls per iteration 	 0.00030199999999999997 us


In [ ]:
el_config = [[[[1], 8.0],
  [[2], 8.0],
  [[3], 8.0],
  [[4], 8.0],
  [[5], 8.0],
  [[6], 8.0],
  [[7], 8.0],
  [[8], 8.0],
  [[9], -0.5354091406692291],
  [[10], -0.30453028190072307],
  [[11], 0.6563010519222342],
  [[12], -0.31076424532492863],
  [[13], -0.5552694286838691],
  [[14], 8.0],
  [[15], 8.0],
  [[16], 8.0],
  [[17], 8.0],
  [[18], 8.0],
  [[19], 8.0],
  [[20], 8.0],
  [[21], 8.0],
  [[22], 8.0],
  [[23], 8.0],
  [[24], 8.0],
  [[25], 8.0],
  [[26], 8.0],
  [[27], 8.0],
  [[28], 8.0],
  [[29], 0.0],
  [[30], 0.0],
  [[31], 0.0],
  [[32], 0.0],
  [[33], 1]]]

In [89]:
import random
# t_view = np.linspace(0, endsim, 10000)
colors = ["#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059", \
          "#FFDBE5", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", \
          "#8FB0FF", "#997D87", "#5A0007", "#809693", "#FFAA92", "#1B4400", \
          "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80", "#61615A", "#BA0900", \
          "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100", \
          "#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", \
          "#013349", "#00846F", "#372101", "#FFB500", "#C2FFED", "#A079BF", \
          "#CC0744", "#C0B9B2", "#C2FF99", "#001E09", "#00489C", "#6F0062"]

# for k, v in v_exp.items():
#     if k not in []: #['N8', 'N9', 'S12', 'S11']:
#         c = colors[random.randint(0, 20)]
#         plt.plot(t_exp*1e6, v.__call__(t_exp), color=c, linestyle='--', marker='')
#         plt.plot(np.linspace(t_exp[0], t_exp[-1], 5000)*1e6, v_ideal[k].__call__(np.linspace(t_exp[0], t_exp[-1], 5000)), color=c)
# plt.ylabel('voltage (V)')
# plt.xlabel('time (us)')
# plt.title('ideal voltages (solid) and filtered voltages (dashed)')


In [111]:
down = 1
R = [[0.0, 0, 51]]
y_span_um = (140, 230)
# y_span_um = (0, 373)
n_y_points = int(1e-6*(y_span_um[1]-y_span_um[0])/1000e-9)+1 # set dy (space resolution)

y_points, dy = np.linspace(y_span_um[0], y_span_um[1], n_y_points, retstep=True)
print(y_points)
y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config[::down]])
xaxis = y_points
yaxis = y_pot
adaptive_yaxis = True
q_min = [y_points[np.where(np.array(y) == min(np.array(y)))][0] for y in y_pot]

# fig, ax = plt.subplots()
# # ax.set_ylim(min(yaxis.flatten()), max(yaxis.flatten()))
# ax.set_ylim(-2, 2)
# line, = ax.plot(xaxis, yaxis[0])

# def animate(i):
#     if adaptive_yaxis:
# #         ax.set_ylim(min(yaxis[i]), max(yaxis[i]))
#         try:
#             ax.set_ylim(yaxis[i][find_peaks(-yaxis[i])[0][0]], max(yaxis[i]))
#         except:
#             ax.set_ylim(min(yaxis[i]), max(yaxis[i]))
#     line.set_ydata(yaxis[i])  # update the data
#     return line,

# #Init only required for blitting to give a clean slate.
# def init():
#     line.set_ydata(np.ma.array(xaxis, mask=True))
#     return line,

# ani = animation.FuncAnimation(fig, animate, np.arange(1, int(len(t_exp)/down)-1), init_func=init,
#     interval=25, blit=True)
# HTML(ani.to_jshtml())



[140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153.
 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167.
 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179. 180. 181.
 182. 183. 184. 185. 186. 187. 188. 189. 190. 191. 192. 193. 194. 195.
 196. 197. 198. 199. 200. 201. 202. 203. 204. 205. 206. 207. 208. 209.
 210. 211. 212. 213. 214. 215. 216. 217. 218. 219. 220. 221. 222. 223.
 224. 225. 226. 227. 228. 229. 230.]


In [ ]:
plt.plot(xaxis, yaxis[-1])

In [91]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

X = y_points[::1]
Y = t_exp*1e6
X, Y = np.meshgrid(X, Y)
Z = y_pot[:,::1]

fig = plt.figure('3d')
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False, rstride=3, cstride=3)
ax.set_xlabel('\n position (um)', linespacing=3)
ax.set_ylabel('\n time (us)', linespacing=3)
ax.set_zlabel('\n potential (V)', linespacing=3)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [96]:
## just a single guess
coeff_final = np.array([vg.compute_total_potential_axes([[0, q0, 51.0]], el_config[-1], printing=False)])

const_map = volt_gen.function_map
x_exp = coeff_final.T[const_map['x']]
y_exp = coeff_final.T[const_map['y']]
a_exp = coeff_final.T[const_map['yy']]
z_exp = coeff_final.T[const_map['z']]
c_exp = coeff_final.T[const_map['yyy']]
d_exp = coeff_final.T[const_map['yyyy']]
q_exp = np.full(len(d_exp), q0)
const_exp = coeff_final.T[const_map['const']]
const_exp = [yaxis[0][int(186-xaxis[0])]]


/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/ITVG/VoltageGeneration.py:999: RuntimeWarning: invalid value encountered in sqrt
  sec_freqs = np.sqrt(2*eigvals*ELEM_CHARGE/(self.m*self.scale**2))/(2*np.pi)


In [116]:
#### Checking the Bucket Potential
def f4(y, a, b, c, d, y0):
    return a*(y-y0)**2 + b + c*(y-y0)**3 + d*(y-y0)**4

def f6(y, a, b, c, d, y0, e, f):
    return a*(y-y0)**2 + b + c*(y-y0)**3 + d*(y-y0)**4 + e*(y-y0)**5 + f*(y-y0)**6

ind=10
a, b = a_ideal[ind], b_ideal[ind]
popt, pcov = curve_fit(f4, xaxis, yaxis[ind], p0=[a, 0, 0, b, 186.5], maxfev=10000)
forwardITVG = [a_exp[-1], const_exp[-1], c_exp[-1], d_exp[-1], q_exp[-1]]
# forwardITVG6 = [a_exp[-1], const_exp[-1], c_exp[-1], d_exp[-1], q_exp[-1], e_exp[-1], f_exp[-1]]


plt.figure('show taylor')
plt.plot(xaxis, yaxis[ind], '.', markersize=15) ## plots potential

xview = np.linspace(xaxis[0], xaxis[-1], 10000)
plt.plot(xview, f4(np.array(xview), *popt), 'r--',
         label='taylor: α=%.2e, const=%.2e, c=%.2e, β=%.2e, y0=%.2e' % tuple(popt)) ## plots Taylor fit

# plt.plot(xview, f4(np.array(xview), *forwardITVG), 'g--',
#          label='forward ITVG: α=%.2e, const=%.2e, c=%.2e, β=%.2e, y0=%.2e' % tuple(forwardITVG)) ## plots forward ITVG
# plt.plot(xview, f6(np.array(xview), *forwardITVG6), 'g--',
#          label='forward ITVG: α=%.2e, const=%.2e, c=%.2e, β=%.2e, y0=%.2e, e=%.2e, f=%.2e' % tuple(forwardITVG6))

plt.plot(xview, a*(xview-q0)**2+b*(xview-q0)**4+yaxis[ind][int(186-xaxis[0])], 'k', label='target') ## plots target potential

plt.legend(loc='best', shadow=1)
plt.xlabel('um')
plt.ylabel('Volts')

error = np.linalg.norm(a*(xaxis-q0)**2+b*(xaxis-q0)**4+yaxis[ind][int(186-xaxis[0])]-yaxis[ind])/len(xaxis)
print('error (V): ', error)

error (V):  0.0010595812141620435


In [121]:
#Plot the three cross sections of the given potential
index = 15
R = [[0.0, 193, 51]]
pseudo_cross_section = []

y_span_um = (140, 230)
n_y_points = int(1e-6*(y_span_um[1]-y_span_um[0])/1000e-9)+1 # set dy (space resolution)

x_span_um = 5
n_x_points = 50
x_points = np.linspace(-x_span_um, x_span_um, n_x_points)

z_span_um = 5
n_z_points = 50
z_points = np.linspace(-z_span_um, z_span_um, n_z_points)

for z_point in z_points:
    test_points = np.array([x_points, np.zeros(n_x_points), np.full(n_x_points, z_point)]).T + R
    xz_exact_potential = volt_gen.compute_potential(test_points, [(k[0][0], k[1]) for k in el_config[index]])
    pseudo_cross_section.append(xz_exact_potential)
    
y_points, dy = np.linspace(y_span_um[0], y_span_um[1], n_y_points, retstep=True)
y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + [[0.0, 0.0, R[0][-1]]]
y_poti = volt_gen.compute_potential(y_test_points, [(el[0][0], el[1]) for el in el_config[index]])

nice_fs = 12    #Nice font size
nice_lw = 4     #Nice linewidth

fig = plt.figure(figsize=(15,6))
plt.subplots_adjust(wspace=0.5)

ax_yslice = fig.add_subplot(1, 2, 1)
ax_yslice.set_xlabel('Y position [um]', size=nice_fs*1.5)
#ax_yslice.set_ylabel('Potenial [V]', size=nice_fs*1.5)
ax_yslice.tick_params(axis='both', which='major', labelsize=nice_fs*0.7)

ax_yslice.plot(y_points, y_poti, '-', c=colors[1], label='DATA')
ax_yslice.grid(True, which="both")

ax_xzslice = fig.add_subplot(1, 2, 2)

ax_xzslice.set_xlabel('X position [um]', size=nice_fs*1.5)
ax_xzslice.set_ylabel('Z position [um]', size=nice_fs*1.5)

extent = [-x_span_um, x_span_um, -z_span_um, z_span_um]

# Plot heatmap
ax_xzslice.imshow(
    pseudo_cross_section,
    extent=extent,
    #vmin = 0.0,
    #vmax = 5.0,
    aspect = 'auto', # Prevent the image from getting stretched out
    interpolation = 'nearest', # ???
    origin = 'lower', # Important to do this so that the first row goes on the bottom
    cmap = 'viridis',
)

#plt.legend(prop={'size': 18})
plt.show()

In [ ]:
#### Plot the radial cross sections at various points
R1 = [[0.0, 170, 51]]
R2 = [[0.0, 186.5, 51]]
R3 = [[0.0, 203, 51]]

x_span_um = (-10, 10)
n_x_points = int(1e-6*(x_span_um[1]-x_span_um[0])/1000e-9)+1
x_points = np.linspace(x_span_um[0], x_span_um[1], n_x_points)

z_span_um = (-9, 9)
n_z_points = int(1e-6*(z_span_um[1]-z_span_um[0])/1000e-9)+1
z_points, dz = np.linspace(z_span_um[0], z_span_um[1], n_z_points, retstep=True)

for r in [R1, R2, R3]:
    plt.figure('x-slice')
    x_test_points = np.array([x_points, np.zeros(n_x_points), np.zeros(n_x_points)]).T + r
    x_pot = np.array([volt_gen.compute_potential(x_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config])
    
    plt.plot(x_points, x_pot[0], marker='.', label=r, )
    plt.ylabel('φ(z) [V]')
    plt.xlabel('x [μm]')
    plt.legend()
    
    plt.figure('z-slice')
    z_test_points = np.array([np.zeros(n_z_points), np.zeros(n_z_points), z_points]).T + r
    z_pot = np.array([volt_gen.compute_potential(z_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config])
#     print(np.gradient(z_pot[0])[int(-z_span_um[0]/dz)])
    
    plt.plot(z_points, z_pot[0], marker='.', label=r, )
    plt.ylabel('φ(z) [V]')
    plt.xlabel('z [μm]')
    plt.legend()



In [ ]:
#### Plot radial minimum vs axial position
R1 = [[0.0, 170, 51]]
R2 = [[0.0, 186.5, 51]]
R3 = [[0.0, 203, 51]]

x_span_um = (-10, 10)
n_x_points = int(1e-6*(x_span_um[1]-x_span_um[0])/1000e-9)+1
x_points = np.linspace(x_span_um[0], x_span_um[1], n_x_points)

z_span_um = (-9, 9)
n_z_points = int(1e-6*(z_span_um[1]-z_span_um[0])/1000e-9)+1
z_points, dz = np.linspace(z_span_um[0], z_span_um[1], n_z_points, retstep=True)

xmin = []
zmin = []
for yi in y_points:
    r = [[0.0, yi, 51]]

    x_test_points = np.array([x_points, np.zeros(n_x_points), np.zeros(n_x_points)]).T + r
    x_pot = np.array([volt_gen.compute_potential(x_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config])
    xmin.append(x_points[np.argmin(x_pot[0])])
    
    z_test_points = np.array([np.zeros(n_z_points), np.zeros(n_z_points), z_points]).T + r
    z_pot = np.array([volt_gen.compute_potential(z_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config])
    zmin.append(z_points[np.argmin(z_pot[0])])
    
plt.plot(y_points, xmin, marker='.', label='minimum in x-slice')
plt.plot(y_points, zmin, marker='.', label='minimum in z-slice')
plt.xlabel('Axial position [μm]')
plt.ylabel('Location of minimum [μm]')
plt.legend()
    

In [ ]:
min_y = [y_points[i] for i in np.argmin(y_pot, axis=1)]
plt.plot(min_y)
dist_exp = min_y

x_span_um = 10
n_x_points = 50
x_points = np.linspace(-x_span_um, x_span_um, n_x_points)

z_span_um = 9
n_z_points = 50
z_points = np.linspace(-z_span_um, z_span_um, n_z_points)

min_z = []
min_x = []
for i in range(len(el_config)):
    R = [[0.0, dist_exp[i], 51.0]]
    pseudo_cross_section = []
    for z_point in z_points:
        test_points = np.array([x_points, np.zeros(n_x_points), np.full(n_x_points, z_point)]).T + R
        xz_exact_potential = volt_gen.compute_potential(test_points, [(k[0][0], k[1]) for k in el_config[i]])
        pseudo_cross_section.append(xz_exact_potential)

    min_index = np.unravel_index(np.argmin(pseudo_cross_section), np.array(pseudo_cross_section).shape) # tuple (z, x)
    min_z.append(z_points[min_index[0]])
    min_x.append(x_points[min_index[1]])
    print('index: ', i, 'z: ', z_points[min_index[0]], 'x: ', x_points[min_index[1]])


In [ ]:
plt.plot(min_z)

In [ ]:
#### Forward ITVG
# q_firstguess = q_min # initial guess for transport
q_firstguess = np.full(len(t_exp), q0) # initial guess for splitting
num_guess = 1

q = []
coeff = []
for n in range(num_guess):
    print('\t'.join(('iter', 't [us]', 'R [um]', 'f [Hz]', 'shift [um]')))
    if not n:
        qg = q_firstguess
    else:
        qg = q_guess
        
    expansion_point = [[[0., q, 51.]] for q in qg]
    
    for i in range(len(t_exp)):
        coeffs = volt_gen.compute_total_potential_axes(expansion_point[i], el_config[i], printing=False)
        coeff.append(coeffs)
        shift = -coeffs[1]/(2*coeffs[6])
        q.append(qg[i] + shift)
        print('\t'.join((str(i), '{:.2f}\t{:.2f}\t{:.0f}\t{:.3f}'.format(t_exp[i]*1e6, qg[i], get_freq_from_alpha(coeffs[6], Sr), shift))))
    q_guess = q[n*len(t_exp):(n+1)*len(t_exp)]

coeff_final = np.array(coeff[len(t_exp)*(num_guess-1):len(t_exp)*num_guess])
q_exp = np.array(q[len(t_exp)*(num_guess-1):len(t_exp)*num_guess])
a_exp = coeff_final.T[6]

In [ ]:
const_map = volt_gen.function_map
x_exp = coeff_final.T[const_map['x']]
y_exp = coeff_final.T[const_map['y']]
z_exp = coeff_final.T[const_map['z']]
c_exp = coeff_final.T[const_map['yyy']]
d_exp = coeff_final.T[const_map['yyyy']]
const_exp = coeff_final.T[const_map['const']]
# plt.plot(t_exp, x_exp)
# plt.plot(t_exp, y_exp)
# plt.plot(t_exp, z_exp)

# plt.plot(t_exp, q_exp)
# q_exp = np.repeat(q0, len(a_exp))


In [ ]:
e_exp = coeff_final.T[const_map['yyyyy']]
f_exp = coeff_final.T[const_map['yyyyyy']]

In [ ]:
import csv

# dist_exp = InterpolatedUnivariateSpline(np.append(t_ideal, 1), np.append(d_ideal, d_ideal[-1]), k=5).__call__(t_exp)
dist_exp = np.array([d_t(t, (t_exp[0], t_exp[-1]), qspan=(d0/2*1e6, 45)) for t in t_exp]) ## um (half the separation)
f_exp = np.sqrt((2*a_exp*1e12 + 3*d_exp*1e24*(dist_exp*2e-6)**2)*ct.e/mass)/(2*pi) # achieved frequency in Hz
ep_exp = ct.e**2/(pi*ct.epsilon_0*mass*(2*pi*f_exp)**2*(dist_exp*2e-6)**3)
f2_exp = np.sqrt((2*pi*f_exp)**2*(1+ep_exp))/(2*pi)
plt.plot(f_exp)
plt.plot(f2_exp)

with open('MB_split_1000kHz_simulated.csv', mode='w') as mb:
    mb_writer = csv.writer(mb, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for a,b,f1,f2 in zip(a_exp, d_exp, f_exp, f2_exp):
        mb_writer.writerow([a, b, f1, f2])


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

xx = coeff_final.T[vg.function_map['xx']][-1]
xy = coeff_final.T[vg.function_map['xy']][-1]
xz = coeff_final.T[vg.function_map['xz']][-1]
yy = coeff_final.T[vg.function_map['yy']][-1]
yz = coeff_final.T[vg.function_map['yz']][-1]
zz = coeff_final.T[vg.function_map['zz']][-1]

hessian = np.array([
    [xx, .5*xy, .5*xz],
    [.5*xy, yy, .5*yz],
    [.5*xz, .5*yz, zz]
])

eigvals, v = sp.linalg.eigh(hessian)
eigvecs = v.T
sec_freqs = np.sqrt(2*eigvals*ct.e/(vg.m*vg.scale**2))/(2*np.pi)

print('xx', get_freq_from_alpha(coeffs[3], Ca))
print('yy', get_freq_from_alpha(coeffs[6], Ca))
print('zz', get_freq_from_alpha(coeffs[8], Ca))

print("Frequencies in Hz:", sec_freqs)
print("Corresponding axes (as row vectors):\n", eigvecs)
print("Each eigenvector in spherical coordinates (theta, phi):\n", \
  (np.arctan2((eigvecs[:,0]**2 + eigvecs[:,1]**2)**0.5, eigvecs[:,2])*180/np.pi),\
     np.arctan2(eigvecs[:,1], eigvecs[:,0])*180/np.pi)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
labels = ['yy', 'xx', 'zz']
for i, k in enumerate(['+x', '+y', '+z']):
    X,Y,Z = 0,0,0
    U,V,W = np.eye(3)[i]
    ax.quiver(X,Y,Z,U,V,W, color=['k']*3, arrow_length_ratio=0.1)
    ax.text(U,V,W, k)
    
for i, p in enumerate(eigvecs):
    X,Y,Z = 0,0,0
    U,V,W = p
    ax.quiver(X,Y,Z,U,V,W, arrow_length_ratio=0.1)
    ax.text(U,V,W, labels[i])
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
ax.set_xlabel('\n x', linespacing=3)
ax.set_ylabel('\n y', linespacing=3)
ax.set_zlabel('\n z', linespacing=3)
plt.show()


In [ ]:
for k, v in const_map.items():
    print(k, '\t', coeff_final[0][v])
    
print('xx', get_freq_from_alpha(coeffs[3], Sr))
print('yy', get_freq_from_alpha(coeffs[6], Sr))
print('zz', get_freq_from_alpha(coeffs[8], Sr))

## Analyze Noise

In [ ]:
plt.rcParams.update({'font.size': 20})

q_delay = q_ideal_func.__call__(np.clip(t_exp-6e-6, 0, None))

plt.figure('position')
plt.plot(t_exp*1e6, q_ideal_func.__call__(t_exp), 'k', label='ideal')
plt.plot(t_exp*1e6, q_delay, 'k--', label='ideal (delayed)')
plt.plot(t_exp*1e6, q_exp, label='experiment')
plt.legend(loc='best', shadow=1)

q_noise = q_exp - q_delay
a_noise = a_exp - a_ideal_func.__call__(t_exp)

# plt.figure('position error (time domain)')
# plt.plot(t_exp*1e6, q_noise)

N = len(t_exp)*1 # zeropadding = sinc interpolation
q_w = np.fft.rfftfreq(N, dx)
ddq_noise = np.gradient(np.gradient(q_noise, dx), dx)

plt.figure('acceleration error (time domain)')
plt.plot(t_exp*1e6, ddq_noise)

ddq_fft = np.fft.rfft(ddq_noise*np.hanning(len(t_exp)), N)
ddq_fftmag = np.abs(ddq_fft)
ddq_fftphs = np.angle(ddq_fft)

plt.figure('acceleration error (frequency domain) magnitude')
plt.semilogy(q_w, ddq_fftmag, marker='.')
peaks = find_peaks(ddq_fftmag)[0]
plt.plot(q_w[peaks], ddq_fftmag[peaks], 'kx')
plt.xlabel('Hz')
print(q_w[peaks])

plt.figure('axial freq')
plt.plot(t_exp*1e6, get_freq_from_alpha(a_ideal_func.__call__(t_exp), m_=Ca)/1e6, 'k', label='ideal')
plt.plot(t_exp*1e6, get_freq_from_alpha(a_exp, m_=Ca)/1e6, label='experiment')
plt.legend(loc='best', shadow=1)
plt.xlabel('us')
plt.ylabel('MHz')


In [ ]:
plt.rcParams.update({'font.size': 20})

def f2(y, a, y0, const):
    return a*(y-y0)**2 + const

def f4(y, a, b, c, d, y0):
    return a*(y-y0)**2 + b + c*(y-y0)**3 + d*(y-y0)**4
    
## Fit the potential with a quartic polynomial
t_fit = np.linspace(t_exp[0], t_exp[-1], len(y_pot))
q_fit = []
a_fit = []
q = q_ideal_func.__call__(t_fit)
a = a_ideal_func.__call__(t_fit)
for i, y in enumerate(y_pot):
    y = np.array(y)
    minindex = np.where(y == min(y))
    interval = [int(minindex-20/dy), int(minindex+20/dy)]
    xdata = y_points[interval[0]:interval[1]]
    ydata = y[interval[0]:interval[1]]
    popt, pcov = curve_fit(f4, xdata, ydata, p0=[a[i], 0, 0, 0, y_points[minindex][0]], maxfev=10000)
    q_fit.append(popt[-1])
    a_fit.append(popt[0])

plt.figure('show taylor')
plt.plot(xdata, ydata, '.', markersize=15)
# forwardITVG = [a_exp[-1], const_exp[-1], c_exp[-1], d_exp[-1], q_exp[-1]]
# forwardITVG = [a_exp[-1], q_exp[-1], const_exp[-1]]
xview = np.linspace(xdata[0], xdata[-1], 10000)
# plt.plot(xview, f4(np.array(xview), *forwardITVG), 'k',
#          label='forward ITVG: a=%.2e, b=%.2e, c=%.2e, d=%.2e, y0=%.2e' % tuple(forwardITVG))
# plt.plot(xview, f2(np.array(xview), *forwardITVG), 'k',)
# plt.plot(xview, pot_grid.__call__(t_exp[-1], xview*1e-6)[0], 'g--')
plt.plot(xview, f4(np.array(xview), *popt), 'r--',
         label='taylor: a=%.2e, b=%.2e, c=%.2e, d=%.2e, y0=%.2e' % tuple(popt))
plt.legend(loc='best', shadow=1)
plt.xlabel('um')
plt.ylabel('Volts')

# plt.figure('position')
# plt.plot(t_fit*1e6, q_fit, label='taylor fit')
# plt.plot(t_fit*1e6, q, 'k', label='ideal')
# plt.plot(t_exp*1e6, q_exp, label='forward ITVG')
# plt.legend(loc='best', shadow=1)
# plt.xlabel('us')
# plt.ylabel('um')

# plt.figure('axial freq')
# plt.plot(t_fit*1e6, get_freq_from_alpha(a_fit, m_=Sr)/1e6, label='taylor fit')
# plt.plot(t_fit*1e6, get_freq_from_alpha(a, m_=Sr)/1e6, 'k', label='ideal')
# plt.plot(t_exp*1e6, get_freq_from_alpha(a_exp, m_=Sr)/1e6, label='forward ITVG')
# plt.legend(loc='best', shadow=1)
# plt.xlabel('us')
# plt.ylabel('MHz')


In [ ]:
index = 46
plt.plot(y_points, y_pot[index], label='compute potential')
plt.plot(y_points, y_exp[index]*(y_points-186.5) + c_exp[index]*(y_points-186.5)**3 + a_exp[index]*(y_points-186.5)**2 + d_exp[index]*(y_points-186.5)**4 + np.repeat(const_exp[index], len(y_points)), label='forward ITVG')
plt.xlabel('um')
plt.ylabel('V')
plt.legend()


# Simulate Ions

**to do:**
- clean this bad boi up!!
- add 'lattice-type' simulation

In [4]:
def heat_sim(tff, q_protocol, t_eval, ms=1E-8, showplot=False, a=None, b=None, g=None, c=None, deg=2):
    ''' simulates ion motion and returns heat gained
        parameters: final integration time, motion protocol, time window for heat calculation 
        all length units must be in um'''
    if a is not None:
        alpha = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), np.append(a*1e12, a[-1]*1e12), k=deg)
    else:
        alpha = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), [get_alpha_from_freq(omega_0/(2*pi))]*(len(t_exp)+1), k=deg)
    if b is not None:
        beta = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), np.append(b*1e24, b[-1]*1e24), k=deg)
    else:
        beta = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), [0]*(len(t_exp)+1), k=deg)
    if g is not None:
        gamma = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), np.append(g*1e6, g[-1]*1e6), k=deg)
    else:
        gamma = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), [0]*(len(t_exp)+1), k=deg)
    if c is not None:
        cubic = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), np.append(c*1e18, c[-1]*1e18), k=deg)
    else:
        cubic = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), [0]*(len(t_exp)+1), k=deg)
    qi = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), np.concatenate((q_protocol/1e6, [q_protocol[-1]/1e6])), k=deg)
    
    def fn(t, v):
        x = v[:Nion]
        dxdt = v[Nion:]
        dist = get_dist_matrix(x)
        
        eom = [] ## equation of motion
        for i in range(Nion): ## iterate over all the ions
            eom.append(-q_ions[i]/m[i]*p_derivative(phi, point=[t, x[i]-qi.__call__(t), alpha.__call__(t), beta.__call__(t), m[i], gamma.__call__(t), cubic.__call__(t)]) +
                       k0*q_ions[i]/m[i]*sum([q_ions[j]*abs(dist[i][j])/(dist[i][j]**3) for j in range(Nion) if j!=i]))
        
        return np.concatenate([dxdt, eom])
        
    outsol = solve_ivp(fn, (0, tff), np.concatenate([x0, x0dot]), method='RK45', dense_output=True, t_eval=t_eval, max_step=ms)    
    h = get_heat(outsol.y[0]-qi.__call__(t_eval), 2*pi*get_freq_from_alpha(alpha.__call__(tff)/1e12, m[0]), m[0])
    
    if showplot:
        t_view = np.linspace(0, tff, int(1e9*tff))
        q = qi.__call__(t_view)
        x = outsol.sol(t_view)[0]

        fig, ax = plt.subplots()
        plt1, = ax.plot(t_view*1e6, q*1e6, 'k--')
#         ax.plot(t_view*1e6, x*1e6)
        ax.set_ylabel('potential minimum position (um)', color='k')
        ax2=ax.twinx()
        plt2, = ax2.plot(t_view*1E6, (x-q)*1E9)
#         plt2, = ax2.plot([], [])
        ax2.set_ylabel('ion oscillation (nm)')
        plt3 = ax2.axvline(t_eval[0]*1E6, color='r', linestyle='--')
        ax2.axvline(t_eval[-1]*1E6, color='r', linestyle='--')
        ax.set_xlabel('time (us)')
        plt.legend((plt1, plt2, plt3), ('potential minimum', 'ion motion', 'heat calculation window'), loc='best')
        
        for n in range(1, Nion):
            x = outsol.sol(t_view)[n]
            ax2.plot(t_view*1E6, (x-q)*1E9)
            
    return outsol, h, outsol.y, qi.__call__(t_eval)

def phi(t, y, a, b, m, g=0, c=0):
    ''' electric potential function [SI Units]
    y is the difference between ion position and potential minimum 
    '''
    return a*y**2 + b*y**4 + g*y + c*y**3

def p_derivative(func, ms=1e-11, var=1, point=[], order=1):
    ''' implements a partial derivative '''
    args = point[:]
    def wraps(x):
        args[var] = x
        return func(*args)
    return derivative(wraps, point[var], dx = ms, n=order)
    
def get_dist_matrix(y):
    ''' gets distance matrix given positions of N ions 
        x = Nion x 1 vector '''
    D=np.zeros((Nion, Nion))
    for i in range(Nion):
        for j in range(i+1, Nion):
            D[i][j]=y[i]-y[j]
            D[j][i]=-D[i][j]
    return D

def get_heat(yvalues, w, m_=40*ct.m_u): ## calculates <n0> (average motional state) Given by Hercul 3.2 eq 97
    maxy = np.max(yvalues)
    miny = np.min(yvalues)
    A = (maxy-miny)/2
    return A**2*m_*np.abs(w)/(2*hbar)

def find_ic(q0, a0, N, xguess, b0=0, disp=False):
    ''' given q, phi coefficients, and num ion, minimize the total energy of the system
    both q and coeff have length scale in um'''
    def total_potential_energy(x, *args):
        q0, a0, b0 = args
        dist = get_dist_matrix(x)
        U = sum([a0*q_ions[i]*(x[i]-q0)**2 + b0*q_ions[i]*(x[i]-q0)**4 + 0.5*k0*q_ions[i]*sum([q_ions[j]/abs(dist[i][j])
                                                                   for j in range(N) if j!=i]) for i in range(N)])
        return U/1.6E-19
    idealx = minimize(total_potential_energy, xguess, args=(q0, a0, b0), method='SLSQP',
                     options={'ftol': 1e-12, 'eps': 1E-12, 'maxiter':1000})
    if disp:
        print(idealx)
    return idealx.x

In [ ]:
hbar = ct.hbar
eps0 = ct.epsilon_0
k0 = 1/(4*pi*eps0)
amu = ct.m_u
elem_charge = ct.e
um = 1e-6

# t_exp = np.linspace(0, 5e-6, 100)
# q_exp = np.full(len(t_exp), 0)
# a_exp = np.full(len(t_exp), get_alpha_from_freq(1e6, m_=86*amu)/1e12)

Nion = 3
q_ions = elem_charge*   np.full(Nion, 1)
m = amu*                np.array([43, 86, 43])
x0dot = um*             np.full(Nion, 0)
x0guess = um*           np.array([q_exp[0]-1, q_exp[0], q_exp[0]+1])

x0 = find_ic(q_exp[0]/1e6, a_exp[0]*1e12, Nion, x0guess)
x0 = x0 * np.array([0.995, 1.001, 1.002])
print(x0*1e6)

In [ ]:
heat_start = 5e-6
heat_end = 55e-6

t_eval = np.linspace(heat_start, heat_end, 4096)
outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_exp, t_eval, showplot=True, a=a_exp, b=None, g=None, c=None, deg=1, ms=1e-8)
print('heat gained (quanta): ', n)

In [ ]:
def heat_sim2(tff, t_eval, ypot_final, ms=1e-8, showplot=False, degx=1, degy=3):
    ''' simulates ion motion and returns heat gained
        parameters: final integration time, motion protocol, time window for heat calculation 
        all length units must be in um'''
    t_sim = np.concatenate((t_exp, [tff*1.1]))
    pot_sim = np.concatenate((ypot_final, ypot_final[-1:, :]), axis=0)
    y_sim = np.concatenate(([-1000e-6], y_points*1e-6, [1000e-6]))
    pot_sim = np.concatenate((pot_sim[:,:1], pot_sim, pot_sim[:,-1:]), axis=1)
    
    pot_func = RectBivariateSpline(t_sim, y_sim, pot_sim, kx=degx, ky=degy)
    
    def fn(t, v):
        x = v[:Nion]
        dxdt = v[Nion:]
        dist = get_dist_matrix(x)
        
        eom = [] ## equation of motion
        for i in range(Nion): ## iterate over all the ions
            eom.append(-q_ions[i]/m[i]*p_derivative(phi2, point=[t, x[i], pot_func]) +
                       k0*q_ions[i]/m[i]*sum([q_ions[j]*abs(dist[i][j])/(dist[i][j]**3) for j in range(Nion) if j!=i]))
        
        return np.concatenate([dxdt, eom])
        
    outsol = solve_ivp(fn, (0, tff), np.concatenate([x0, x0dot]), method='RK45', dense_output=True, t_eval=t_eval, max_step=ms)    
    
    if showplot:
        t_view = np.linspace(0, tff, int(1e9*tff))
        x = outsol.sol(t_view)[0]
        plt.plot(t_view*1e6, x*1e6)
        plt.plot(t_view*1e6, outsol.sol(t_view)[1]*1e6)
        plt.plot(t_view*1e6, outsol.sol(t_view)[2]*1e6)
        plt.plot(t_view*1e6, outsol.sol(t_view)[3]*1e6)
#         plt.ylabel('ion position (um)', color='k')
        
    return outsol, outsol.y, pot_func

def phi2(t, y, pot_func):
    ''' electric potential function [SI Units]
    y is the difference global position 
    '''
    return float(pot_func.__call__(t, y))

def find_ic2(ypot, N, xguess, disp=False):
    ''' given q, phi coefficients, and num ion, minimize the total energy of the system
    both q and coeff have length scale in m'''
    def total_potential_energy(x, *args):
        ypot_func = args[0]
        dist = get_dist_matrix(x)
        U = sum([q_ions[i]*ypot_func.__call__(x[i]) + 0.5*k0*q_ions[i]*sum([q_ions[j]/abs(dist[i][j])
                                                                   for j in range(N) if j!=i]) for i in range(N)])
        return U/1.6e-19
    idealx = minimize(total_potential_energy, xguess, args=(ypot), method='Nelder-Mead', options={'xatol': 1e-9})
    if disp:
        print(idealx)
    return idealx.x

In [ ]:
heat_start = 305e-6
heat_end = 315e-6

t_eval = np.linspace(heat_start, heat_end, 4096)
outsol, ion_eval, pot_grid = heat_sim2(t_eval[-1], t_eval, y_pot, showplot=True, ms=1e-8)
# endfreq = get_freq_from_alpha(a_exp[-1], Ca)
endfreq = f0
print(endfreq, 'Hz')
n = get_heat(ion_eval[0], 2*pi*endfreq, Ca)
print('heat gained (quanta): ', n)

# Investigations

## Discretization Investigation

In [ ]:
#### Discretization Investigation ####
endbuff = 0
# dt_vec = np.logspace(-7.25, -6.75, 51)
# dt_vec = np.linspace(100e-9, 1000e-9, 51)
dt_vec = 5000e-9/np.linspace(50, 500, 46)

heat_play = []
for dt in dt_vec:
    d = defaultdict(list)
    for f in allvoltages:
        [d[k].append(v) for k, v in f.items()]

    buffer1 = 2
    buffer2 = 2
    Ntot = int(buffer1+round(T0/dt)+buffer2)
    Ttot = Ntot*dt
#     buffer1 = (Ttot-T0)/2 ## buffers now have units of time instead of samples
#     buffer2 = (Ttot-T0)/2
    buffer1 = 2*dt ## buffers now have units of time instead of samples
    buffer2 = Ttot-T0-buffer1

    v_ideal = {} # voltages from backwards ITVG
    v_DAC = {} # sampled voltages for DAC commands
    for k, v in d.items():
        tmiddle = np.linspace(buffer1, T0+buffer1, int(T0*1000e6))
        vmiddle = InterpolatedUnivariateSpline(np.linspace(buffer1, T0+buffer1, len(v)), v, k=3).__call__(tmiddle)
        vevents = np.concatenate(([v[0]], vmiddle, [v[-1]]))
        tevents = np.concatenate(([-0.1e-6], tmiddle, [Ttot+0.1e-6]))
        v_ideal[k] = interp1d(np.append(tevents, 1), np.append(vevents, vevents[-1]))
        v_DAC[k] = v_ideal[k].__call__([(n+endbuff)*dt for n in range(Ntot)])
    endbuffer = 3e-6/dt
    endsim = dt*(len(v_DAC['N9']) + endbuffer)
    
    t_all = []
    v_exp = {}
    for k, v in v_DAC.items():
        tevents, vevents = set_vinput(v, dt, st, slew_rate)
        v_exp[k] = interp1d(np.concatenate([tevents, [1]]), np.concatenate([vevents, vevents[-1:]]))
        t_all = np.concatenate([t_all, tevents])
    t_all = np.unique(sorted(t_all))
    
    endITVG = endsim - 0e-6 # we can't find the potential for voltages that don't exist! hence the minus 0.1us
    volt_gen = vg
    N_final = int(endITVG/50e-9)+1 # set dt (time resolution)
    t_exp, dx = np.linspace(0, (endITVG), N_final, retstep=True)
    t_exp = np.unique(sorted(np.concatenate((t_all, t_exp))))
    print('# of potential solves:', len(t_exp), ' dt [us]:', dt, 'Ntot:', Ntot, ' end sim [us]:', endITVG, ' buffer1:', buffer1)

#     for k, v in v_exp.items():
#         if k in ['N8']:
#             c = colors[random.randint(0, 20)]
#             plt.plot(t_exp*1e6, v.__call__(t_exp), color=c, linestyle='--', marker='.')
#             plt.plot(np.linspace(t_exp[0], t_exp[-1], 10000)*1e6, v_ideal[k].__call__(np.linspace(t_exp[0], t_exp[-1], 10000)), color=c)
#     plt.ylabel('voltage (V)')
#     plt.xlabel('time (us)')
#     plt.title('ideal voltages (solid) and sampled voltages (dashed)')
#     plt.show()

    #### calculate electrostatic potential
    bigarray = np.array([v.__call__(t_exp) for k, v in v_exp.items()]).T
    el_config = []
    for row in bigarray:
        el_configi = [[[i+1], v] for i, v in enumerate(row)]
        el_configi.extend([[[el], 1] for el in volt_gen.rf_electrodes])
        el_config.append(el_configi)
    R = [[0.0, 0, 50.0]]
    y_span_um = (0, 373)
    n_y_points = int(1e-6*(y_span_um[1]-y_span_um[0])/1000e-9) # set dy (space resolution)
    y_points, dy = np.linspace(y_span_um[0], y_span_um[1], n_y_points, retstep=True)
    y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
    y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config])
    
    #### simulation ion
    heat_start = endITVG - 0.5e-6
    heat_end = endITVG
    t_eval = np.linspace(heat_start, heat_end, 4096)
    outsol, ion_eval, pot_grid = heat_sim2(t_eval[-1], t_eval, y_pot, showplot=False, ms=1e-9)
    endfreq = get_freq_from_alpha(a_exp[-1], Sr)
    n = get_heat(ion_eval[0], 2*pi*endfreq, Sr)
    print('heat gained (quanta): ', n)
    heat_play.append(n)
    
heat_begin = heat_play


In [ ]:
plt.figure('heat')
plt.hlines(0.032, dt_vec[0]*1e9, dt_vec[-1]*1e9)
plt.semilogy(dt_vec*1e9, heat_play, label='beginning', marker='.')
# plt.plot(dt_vec*1e9, heat_middle, label='middle', marker='.')
# plt.plot(dt_vec*1e9, heat_endd, label='end', marker='.')
plt.xlabel('sampling period (ns)')
plt.ylabel('heat gained after 5us transport (quanta)')
plt.legend(loc='best', shadow=1)

In [ ]:
#### Discretization Investigation Pt. 2 ####
T_vec = np.linspace(5e-6, 100e-6, 96)
T_vec_smaller = np.linspace(5e-6, 100e-6, 20)

# #### Perfect STA
# f0 = 4e6 # Axial frequency [Hz]
# T = 5e-6 # Transport time [s]
# move = (141.5, 186.5) # Start and End positions [um]
# a0 = get_alpha_from_freq(f0, m_=Sr)*1e-12 # Specify ion mass. [Units are V/m^2, but ITVG uses um as the length scale]

# t_init = np.linspace(0, T+10e-6, 1024)
# q_init = np.array([qsta2(t, (0, T), qspan=move, f=T*f0) for t in t_init])
# q_init_func = InterpolatedUnivariateSpline(t_init/T, q_init, k=3) ## scale time to be unitless

# heat_play = []
# for T in T_vec:
#     t_exp = np.linspace(0, T+10e-6, len(t_init))
#     q_exp = q_init_func.__call__(t_exp/T)
#     a_exp = np.repeat(a0, len(t_exp))
    
#     #### simulation ion
#     Nion = 1
#     q_ions = elem_charge*   np.full(Nion, 1)
#     m = amu*                np.array([88])
#     x0dot = um*             np.full(Nion, 0)
#     x0guess = um*           np.array([q_exp[0]])
#     x0 = find_ic(q_exp[0]/1e6, a_exp[0]*1e12, Nion, x0guess)
    
#     heat_start = t_exp[-1]-5e-6
#     heat_end = t_exp[-1]
#     t_eval = np.linspace(heat_start, heat_end, 4096)
#     outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_exp, t_eval, showplot=False, a=a_exp, b=None, g=None, c=None, deg=2, ms=1e-8)
#     print('transport period: ', T, ' heat gained: ', n)
#     heat_play.append(n)

#### Sampled STA
dt = 0.125*1e-6

heat_play = []
for T, allvoltages in zip(T_vec_smaller, allvoltages_vec):
#     t_ideal = np.linspace(0, T0, 100)
#     q_ideal = np.array([qsta2(t, (0, T0), qspan=move, f=T0*f0) for t in t_ideal])
#     a_ideal = np.repeat(a0, len(t_ideal))
#     tilt_angle = np.radians(15) # specify tilt [rad]
#     volt_gen = vg
#     bounds = (-40.0, 40.0)
#     norm = np.full((6, 1), 1) # normalization factor to pull out certain coefficients
#     norm[[1,4]] = 1e2 # y and yy coefficients
#     # norm[[3,5,8]] = 1e-9 # tilt coefficients
#     volt_gen.set_electrode_grouping(volt_gen.electrode_grouping) # sets previous voltages to None

#     allvoltages = []
#     for q, a, i in zip(q_ideal, a_ideal, 1+np.arange(len(t_ideal))):
#         print(i, 'solving at (q, a):', q, a)
#         R = [[0, q, 50.0]] # trap center
#     #     notconstrained = [29,30,31,32]
#         notconstrained = []
#     #     notconstrained.extend([k for k, v in MB_intervals.items() if getOverlap([q-67.6,q+67.6], v)]) # we can tune this parameter
#         notconstrained = [10,11,12,13]
#     #     notconstrained = [10, 11, 4, 5, 6, 7, 14, 15, 16, 17]
#         print('unconstrained electrodes: ', notconstrained)
#         constrained_voltages = [(i[0]-1, 8) for i in electrode_grouping if i[0] not in notconstrained]
#         xx, xz, zz = tiltWithPseudo(volt_gen, R, tilt_angle, a, volt_gen.rf_electrodes, debug=False)
#         cons = [('x', 0), ('y', 0), ('z', 0), ('yy', a), ('xy', 0), ('yz', 0)]
#         nom_v = volt_gen.findControlVoltages(R, cons=[cons], tol=1e-18, fixed_voltages=constrained_voltages, bnds=bounds, normvec=norm, epss=1e-12, independent=False)
#         allvoltages.append({electrode_ordering[num-1]: v for [num, v] in volt_gen.ungroup_configuration(nom_v)})

    d = defaultdict(list)
    for f in allvoltages:
        [d[k].append(v) for k, v in f.items()]

    buffer1 = 2
    buffer2 = 2
    Ntot = int(buffer1+round(T/dt)+buffer2)
    Ttot = Ntot*dt
    buffer1 = 2*dt ## buffers now have units of time instead of samples
    buffer2 = Ttot-T-buffer1

    v_ideal = {} # voltages from backwards ITVG
    v_DAC = {} # sampled voltages for DAC commands
    for k, v in d.items():
        tmiddle = np.linspace(buffer1, T+buffer1, int(T*1000e6))
        vmiddle = InterpolatedUnivariateSpline(np.linspace(buffer1, T+buffer1, len(v)), v, k=3).__call__(tmiddle)
        vevents = np.concatenate(([v[0]], vmiddle, [v[-1]]))
        tevents = np.concatenate(([-0.1e-6], tmiddle, [Ttot+0.1e-6]))
        v_ideal[k] = interp1d(np.append(tevents, 1), np.append(vevents, vevents[-1]))
        v_DAC[k] = v_ideal[k].__call__([(n)*dt for n in range(Ntot)])
    endbuffer = 3e-6/dt
    endsim = dt*(len(v_DAC['N9']) + endbuffer)
    
#     t_all = []
#     v_exp = {}
#     for k, v in v_DAC.items():
#         tevents, vevents = set_vinput(v, dt, st, slew_rate)
#         v_exp[k] = interp1d(np.concatenate([tevents, [1]]), np.concatenate([vevents, vevents[-1:]]))
#         t_all = np.concatenate([t_all, tevents])
#     t_all = np.unique(sorted(t_all))
    
    for k, v in v_ideal.items():
        t_sample = np.linspace(0, tevents[-1], 100)
        v_exp[k] = interp1d(np.concatenate([t_sample, [1]]), np.concatenate([v.__call__(t_sample), [v.__call__(t_sample[-1])]]))
    
    endITVG = endsim - 0e-6 # we can't find the potential for voltages that don't exist! hence the minus 0.1us
    volt_gen = vg
    N_final = int(endITVG/100e-9)+1 # set dt (time resolution)
    t_exp, dx = np.linspace(0, (endITVG), N_final, retstep=True)
#     t_exp = np.unique(sorted(np.concatenate((t_all, t_exp))))
    print('# of potential solves:', len(t_exp), ' dt [us]:', dt, 'Ntot:', Ntot, ' end sim [us]:', endITVG, ' buffer1:', buffer1)

#     for k, v in v_exp.items():
#         if k in ['N8']:
#             c = colors[random.randint(0, 20)]
#             plt.plot(t_exp*1e6, v.__call__(t_exp), color=c, linestyle='--', marker='.')
#             plt.plot(np.linspace(t_exp[0], t_exp[-1], 10000)*1e6, v_ideal[k].__call__(np.linspace(t_exp[0], t_exp[-1], 10000)), color=c)
#     plt.ylabel('voltage (V)')
#     plt.xlabel('time (us)')
#     plt.title('ideal voltages (solid) and sampled voltages (dashed)')
#     plt.show()

    #### calculate electrostatic potential
    bigarray = np.array([v.__call__(t_exp) for k, v in v_exp.items()]).T
    el_config = []
    for row in bigarray:
        el_configi = [[[i+1], v] for i, v in enumerate(row)]
        el_configi.extend([[[el], 1] for el in volt_gen.rf_electrodes])
        el_config.append(el_configi)
    R = [[0.0, 0, 50.0]]
    y_span_um = (0, 373)
    n_y_points = int(1e-6*(y_span_um[1]-y_span_um[0])/1000e-9) # set dy (space resolution)
    y_points, dy = np.linspace(y_span_um[0], y_span_um[1], n_y_points, retstep=True)
    y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
    y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config])
    
    #### simulation ion
    heat_start = endITVG - 2e-6
    heat_end = endITVG
    t_eval = np.linspace(heat_start, heat_end, 4096)
    outsol, ion_eval, pot_grid = heat_sim2(t_eval[-1], t_eval, y_pot, showplot=False, ms=1e-8)
    endfreq = get_freq_from_alpha(a_exp[-1], Sr)
    n = get_heat(ion_eval[0], 2*pi*endfreq, Sr)
    print('heat gained (quanta): ', n)
    heat_play.append(n)
    
heat_4MHz_unsampled = heat_play


In [ ]:
plt.figure('heat vs transport time (STA)')
plt.semilogy(T_vec_smaller*1e6, heat_4MHz_unsampled, label='4MHz unsampled', marker='.')
# plt.semilogy(T_vec*1e6, heat_4MHz_0_175us_sampled, label='4MHz sampled at dt=0.175us (lazy method)', marker='.')
# plt.semilogy(T_vec_smaller*1e6, heat_4MHz_0_175us_sampled_legit, label='4MHz sampled at dt=0.175us', marker='.')
plt.semilogy(T_vec[::5]*1e6, np.array(heat_4MHz_0_3us_sampled[::5])/heat_4MHz_unsampled, label='4MHz sampled at dt=0.3us (lazy method)', marker='.')
plt.semilogy(T_vec_smaller*1e6, np.array(heat_4MHz_0_3us_sampled_legit)/heat_4MHz_unsampled, label='4MHz sampled at dt=0.3us', marker='.')
# plt.semilogy(T_vec*1e6, heat_4MHz_0_25us_sampled, label='4MHz sampled at dt=0.25us', marker='.')
plt.semilogy(T_vec[::5]*1e6, np.array(heat_4MHz_0_125us_sampled[::5])/heat_4MHz_unsampled, label='4MHz sampled at dt=0.125us (lazy method)', marker='.')
plt.semilogy(T_vec_smaller*1e6, np.array(heat_4MHz_0_125us_sampled_legit)/heat_4MHz_unsampled, label='4MHz sampled at dt=0.125us', marker='.')
plt.xlabel('transport time (us)')
plt.ylabel('phonon')
plt.legend(loc='best', shadow=1)

## Time-Dependent Axial Frequency Investigation

In [ ]:
#### Time-Dependent Axial Frequency Investigation ####
f0 = 4e6 # Axial frequency [Hz]
# f0 = 1.41e6
T0 = 2e-6 # Transport time [s]
# T0 = 0.89e-6
move = (140, 190) # Start and End positions [um]
# move = (0, 0.4)
a0 = get_alpha_from_freq(f0, m_=Sr)*1e-12 # Specify ion mass. [Units are V/m^2, but ITVG uses um as the length scale]
# a0 = get_alpha_from_freq(f0, m_=Ca)*1e-12
fnoise = 1/T0

t_exp = np.linspace(0, T0, 1000)
q_exp = np.array([qsta(t, (0, T0), qspan=move, f=T0*f0) for t in t_exp])
# q_exp = q_robust2(t_exp, (0, T0), [f0, f0], qspan=move)
# q_exp = savgol_filter(q_exp, 17, 1)
a_exp = np.repeat(a0, len(t_exp))

# freq_vec = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])*fnoise
freq_vec = np.linspace(0, 2e6, 100)
heat_play = []
for f in freq_vec:
    a_exp = a0*(1+np.array([0.10*np.sin((t-0e-6)*2*np.pi*f) if t<=T0+0e-6 and t>=0e-6 else 0 for t in t_exp]))
#     plt.plot(t_exp*1e6, a_exp)

    hbar = ct.hbar
    eps0 = ct.epsilon_0
    k0 = 1/(4*pi*eps0)
    amu = ct.m_u
    elem_charge = ct.e
    um = 1e-6

    Nion = 1
    q_ions = elem_charge*   np.full(Nion, 1)
    m = amu*                np.array([88])
    x0dot = um*             np.full(Nion, 0)
    x0guess = um*           np.array([q_exp[0]])
    x0 = find_ic(q_exp[0]/1e6, a_exp[0]*1e12, Nion, x0guess)

    heat_start = 7.0e-6
    heat_end = 9.0e-6

    t_eval = np.linspace(heat_start, heat_end, 4096)
    outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_exp, t_eval, showplot=False, a=a_exp, b=None, g=None, c=None, deg=1, ms=1e-9)
    print('heat gained (quanta): ', n)
    
    heat_play.append(n)
heat_10_poly = heat_play
    

In [ ]:
# plt.semilogy(freq_vec, heat_10, marker='.', color='tab:blue', label='amplitude = 0.1*ω0')
plt.semilogy(freq_vec, heat_10_poly, marker='.', ls='--', color='tab:blue', label='amplitude = 0.1*ω0 (polynomial STA)')
# plt.semilogy(freq_vec, heat_25, marker='.', color='tab:orange', label='amplitude = 0.25*ω0')
# plt.semilogy(freq_vec, heat_25_poly, marker='.', color='tab:orange', ls='--', label='amplitude = 0.25*ω0 (polynomial STA)')
# plt.semilogy(freq_vec, heat_50, marker='.', color='tab:green', label='amplitude = 0.5*ω0')
# plt.semilogy(freq_vec, heat_50_poly, marker='.', color='tab:green', ls='--', label='amplitude = 0.5*ω0 (polynomial STA)')

plt.xlabel('frequency of axial frequency noise')
plt.ylabel('heat (quanta)')
plt.title('2us STA 5th-order transport (4MHz, 50um)')
plt.legend(loc='best', shadow=1)

In [ ]:
#### Time-Dependent Axial Frequency Investigation pt 2####
f0 = 4e6 # Axial frequency [Hz]
T0 = 2e-6 # Transport time [s]
move = (140, 190) # Start and End positions [um]
a0 = get_alpha_from_freq(f0, m_=Sr)*1e-12 # Specify ion mass. [Units are V/m^2, but ITVG uses um as the length scale]
fnoise = 1/T0

T_vec = np.linspace(1e-6, 100e-6, 101)
heat_play = []
for Ti in T_vec:
    t_exp = np.linspace(0, Ti, 1000)
    q_exp = np.array([qsta(t, (0, Ti), qspan=move, f=Ti*f0) for t in t_exp])
    a_exp = a0*(1+np.array([0.10*np.sin((t-0e-6)*2*np.pi*100e3) if t<=Ti+0e-6 and t>=0e-6 else 0 for t in t_exp]))
#     plt.plot(t_exp*1e6, q_exp)

    hbar = ct.hbar
    eps0 = ct.epsilon_0
    k0 = 1/(4*pi*eps0)
    amu = ct.m_u
    elem_charge = ct.e
    um = 1e-6

    Nion = 1
    q_ions = elem_charge*   np.full(Nion, 1)
    m = amu*                np.array([88])
    x0dot = um*             np.full(Nion, 0)
    x0guess = um*           np.array([q_exp[0]])
    x0 = find_ic(q_exp[0]/1e6, a_exp[0]*1e12, Nion, x0guess)

    heat_start = Ti + 5e-6
    heat_end = Ti + 10e-6

    t_eval = np.linspace(heat_start, heat_end, 4096)
    outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_exp, t_eval, showplot=False, a=a_exp, b=None, g=None, c=None, deg=1, ms=1e-9)
    print('heat gained (quanta): ', n)
    
    heat_play.append(n)
heat_10_poly = heat_play
    

In [ ]:
plt.semilogy(T_vec[:71]*1e6, heat_play, marker='.', ls='--', color='tab:blue', label='sinusoidal noise. A=0.1*ω0, f=100kHz')

plt.xlabel('transport time (us)')
plt.ylabel('heat (quanta)')
plt.title('STA 5th-order transport (4MHz, 50um)')
plt.legend(loc='best', shadow=1)

In [ ]:
def q_robust2(t, tspan, f, qspan=(0, 120)):
    ''' t is a time vector
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti, dt = tf-t0, t-t0, t[1]-t[0]
    s = np.clip(ti/T, 0, 1)
    
    N = len(f)
    norm = (yf-y0)*(2*N+1)/(T**2*np.prod([(2*np.pi*fi)**2 for fi in f])*sp.special.beta(2*N+2, 2*N+2))
    g = lambda t: norm* (t/T)**(2*N)* (1-(t/T))**(2*N) *(1-2*(t/T))

    xddot = np.array([derivative(g, t, dx=0.05*dt, n=4, order=5) for t in ti])\
            + np.sum([(2*np.pi*fi)**2 for fi in f])*np.array([derivative(g, t, dx=0.05*dt, n=2, order=5) for t in ti])\
            + np.prod([(2*np.pi*fi)**2 for fi in f])*g(ti)
    x_t = np.c_[np.r_[0, xddot[1:-1:2].cumsum()], xddot[::2].cumsum() - xddot[0] / 2].ravel()[:len(xddot)]
    x_t = np.c_[np.r_[0, x_t[1:-1:2].cumsum()], x_t[::2].cumsum() - x_t[0] / 2].ravel()[:len(x_t)]
    x_t = (yf-y0)*x_t/x_t[-1] + y0
    return x_t

def q_robust(t, tspan, f, qspan=(0, 120)):
    ''' t is a time vector
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti, dt = tf-t0, t-t0, t[1]-t[0]
    s = np.clip(ti/T, 0, 1)
    
    N = len(f)
    norm = (yf-y0)*(2*N+1)/(T**2*np.prod([(2*np.pi*fi)**2 for fi in f])*sp.special.beta(2*N+2, 2*N+2))
    g = lambda s: norm*s**(2*N)*(1-s)**(2*N)*(1-2*s)

    xddot = np.array([derivative(g, si, n=2) for si in s])\
            + np.sum([(2*np.pi*fi)**2 for fi in f])*g(s)
    x_t = dt*np.c_[np.r_[0, xddot[1:-1:2].cumsum()], xddot[::2].cumsum() - xddot[0] / 2].ravel()[:len(xddot)]
    x_t = dt*np.c_[np.r_[0, x_t[1:-1:2].cumsum()], x_t[::2].cumsum() - x_t[0] / 2].ravel()[:len(x_t)]
    x_t = (yf-y0)*x_t/x_t[-1] +  y0
    return x_t

f0 = 1.41e6 # Axial frequency [Hz]
f0 = 4e6
T0 = 0.89e-6 # Transport time [s]
T0 = 5e-6
move = (0, 0.4) # Start and End positions [um]
move = (141.5, 186.5)
a0 = get_alpha_from_freq(f0, m_=Ca)*1e-12 # Specify ion mass. [Units are V/m^2, but ITVG uses um as the length scale]
a0 = get_alpha_from_freq(f0, m_=Sr)*1e-12
fnoise = 1/T0

t_exp = np.linspace(0, T0, 1000)
q_exp = q_robust2(t_exp, (0, T0), [f0, f0], qspan=move)
a_exp = np.repeat(a0, len(t_exp))

plt.plot(t_exp*1e6, q_exp, marker='', label='2-point')
# plt.plot(t_exp*1e6, savgol_filter(q_exp, 17, 1), 'k')
plt.xlabel('us')
plt.ylabel('um')
plt.legend()
# plt.plot(t_exp*1e6, q_robust(t_exp, (0, T0), [f0, f0], qspan=move)-q_robust(t_exp, (0, T0), [f0*1.5, f0*0.5], qspan=move))


## Evenly-Spaced Chains

In [ ]:
#### SEXTIC POTENTIAL ####
import random
colors = ["#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059", \
          "#FFDBE5", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", \
          "#8FB0FF", "#997D87", "#5A0007", "#809693", "#FFAA92", "#1B4400", \
          "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80", "#61615A", "#BA0900", \
          "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100", \
          "#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", \
          "#013349", "#00846F", "#372101", "#FFB500", "#C2FFED", "#A079BF", \
          "#CC0744", "#C0B9B2", "#C2FF99", "#001E09", "#00489C", "#6F0062"]

def get_chain_spacing(a, b, h, N, xguess, disp=False):
    ''' minimize the total energy of the system
    SI units'''
    def total_potential_energy(x, *args):
        a0, b0, h0 = args
        dist = get_dist_matrix(np.array(sorted(x)))
        U = sum([a0*ct.e*x[i]**2 + b0*ct.e*x[i]**4 + h0*ct.e*x[i]**6 +
                 0.5*k0*ct.e*sum([ct.e/abs(dist[i][j]) for j in range(N) if j!=i]) for i in range(N)])
        return U/1.6e-19
    idealx = minimize(total_potential_energy, xguess, args=(a, b, h), method='SLSQP',
                     options={'ftol': 1e-27, 'eps': 1e-9, 'maxiter':1000})
    
    positions = np.array(sorted(idealx.x))
    if disp:
        c = colors[random.randint(0, 20)]
        y_plot = np.linspace(2*positions[0], 2*positions[-1], 100)
        plt.plot(y_plot, a*y_plot**2 + b*y_plot**4 + h*y_plot**6, color=c)
        plt.plot(positions, [0]*len(positions), 's', color=c)
        plt.xlabel('axial position (m)')
        plt.ylabel('voltage (V)')
    return positions[1:] - positions[:-1], positions

def cost(abh, *args):
    a,b,h = abh
    Nion = args[0]
    x0guess = args[1]
    d0 = args[2]
    
    chain = get_chain_spacing(a*1e12, b*1e24, h*1e36, Nion, x0guess, disp=False)[0]*1e6
    print((a, b, h), chain, '\n')
    return la.norm(chain - np.repeat(d0, len(chain)))



In [ ]:
#### ALL LENGTH UNITS ARE METERS (SI UNITS) ####
hbar = ct.hbar
eps0 = ct.epsilon_0
k0 = 1/(4*pi*eps0)
amu = ct.m_u
elem_charge = ct.e
um = 1e-6

d0 = 5 # um
Nion = 4
x0guess = um*np.array([-1.5*d0, -.5*d0, +.5*d0, +1.5*d0])

ideal_ab = minimize(cost, [1e-6, 1e-9, 1e-10], args=(Nion, x0guess, d0), method='Nelder-Mead')
print(ideal_ab)


In [ ]:
#### VIEW OUTPUT ####
abh = ideal_ab.x
a,b,h = abh
print('(α, β, η): ', a,b,h)
positions = get_chain_spacing(a*1e12, b*1e24, h*1e36, Nion, x0guess, disp=True)[1]
print('ion positions: ', positions)

In [ ]:
#### QUARTIC POTENTIAL
import random
colors = ["#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059", \
          "#FFDBE5", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", \
          "#8FB0FF", "#997D87", "#5A0007", "#809693", "#FFAA92", "#1B4400", \
          "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80", "#61615A", "#BA0900", \
          "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100", \
          "#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", \
          "#013349", "#00846F", "#372101", "#FFB500", "#C2FFED", "#A079BF", \
          "#CC0744", "#C0B9B2", "#C2FF99", "#001E09", "#00489C", "#6F0062"]

def get_chain_spacing(a, b, N, xguess, disp=False):
    ''' minimize the total energy of the system
    SI units'''
    def total_potential_energy(x, *args):
        a0, b0 = args
        dist = get_dist_matrix(np.array(sorted(x)))
        U = sum([a0*ct.e*x[i]**2 + b0*ct.e*x[i]**4 +
                 0.5*k0*ct.e*sum([ct.e/abs(dist[i][j]) for j in range(N) if j!=i]) for i in range(N)])
        return U/1.6e-19
    idealx = minimize(total_potential_energy, xguess, args=(a, b), method='SLSQP',
                     options={'ftol': 1e-27, 'eps': 1e-9, 'maxiter':1000})
    
    positions = np.array(sorted(idealx.x))
    if disp:
        c = colors[random.randint(0, 20)]
        y_plot = np.linspace(2*positions[0], 2*positions[-1], 100)
        plt.plot(y_plot, a*y_plot**2 + b*y_plot**4, color=c)
        plt.plot(positions, [0]*len(positions), 's', color=c)
        plt.xlabel('axial position (m)')
        plt.ylabel('voltage (V)')
    return positions[1:] - positions[:-1], positions

def cost(ab, *args):
    a,b= ab
    Nion = args[0]
    x0guess = args[1]
    d0 = args[2]
    
    chain = get_chain_spacing(a*1e12, b*1e24, Nion, x0guess, disp=False)[0]*1e6
    print((a, b), chain, '\n')
    return la.norm(chain - np.repeat(d0, len(chain)))



In [ ]:
#### ALL LENGTH UNITS ARE METERS (SI UNITS) ####
hbar = ct.hbar
eps0 = ct.epsilon_0
k0 = 1/(4*pi*eps0)
amu = ct.m_u
elem_charge = ct.e
um = 1e-6

d0 = 10 # um
Nion = 4
x0guess = um*d0*np.array([-1.5, -0.5, 0.5, 1.5])

ideal_ab = minimize(cost, [1e-6, 1e-9], args=(Nion, x0guess, d0), method='Nelder-Mead')
print(ideal_ab)

In [ ]:
#### VIEW OUTPUT ####
ab = ideal_ab.x
a,b = ab
print('(α, β): ', a,b)
positions = get_chain_spacing(a*1e12, b*1e24, Nion, x0guess, disp=True)[1]
print('ion positions: ', positions)

In [ ]:
#### Spectroscopy on this bucket potential        
t_exp = np.linspace(0, 5e-6, 50)
# y_span_um = (-100, 100)
y_span_um = (120, 250)
n_y_points = int(1e-6*(y_span_um[1]-y_span_um[0])/1000e-9)+1 # set dy (space resolution)
y_points, dy = np.linspace(y_span_um[0], y_span_um[1], n_y_points, retstep=True)
# y_poti = np.array([a*y_points**2 + b*y_points**4]*len(t_exp))
y_poti = np.array([y_pot[0]]*len(t_exp))

Nion = 4
q_ions = ct.e*   np.full(Nion, 1)
m = amu*                np.array([40, 40, 40, 40])
x0dot = um*             np.full(Nion, 0)

ymin = y_points[np.argmin(y_poti, axis=1)[0]]
x0guess = um*d0*np.array([-1.5, -0.5, 0.5, 1.5]) + 186.5e-6
print(x0guess)
x0 = find_ic2(InterpolatedUnivariateSpline(y_points*1e-6, y_poti[0], k=3), Nion, x0guess, disp=False)
x0 *= np.array([1.0025, 1.0015, 0.9995, 0.9985])
print(x0*1e6)

heat_start = 5e-6
heat_end = 45e-6
t_eval = np.linspace(heat_start, heat_end, 4096)
outsol, ion_eval, pot_grid = heat_sim2(t_eval[-1], t_eval, y_poti, showplot=True, ms=1e-8)
plt.figure('potential')
plt.plot(y_points, y_poti[0])

N = 2**16
dx = t_eval[1]-t_eval[0]
ion_w = [np.fft.rfftfreq(N, dx)]*Nion
ion_fft = [np.fft.rfft(ion_eval[i]-np.average(ion_eval[i]), N) for i in range(Nion)]  

plt.figure('freq domain sim2')
ion_fft = np.abs(ion_fft)
ion_fft_avg = np.mean(ion_fft, axis=0)
ion_fft_avg /= np.max(ion_fft_avg)
plt.plot(ion_w[0], ion_fft_avg)

## find peaks in spectroscopy
peak_indices = find_peaks(ion_fft_avg, 0.05, distance=100)[0]
peak_w = ion_w[0][peak_indices]
print(peak_w)



## HVAmp

In [ ]:
#### Fitting a linear model for the HVamp
import pandas as pd
from scipy.stats.distributions import chi2

fig = plt.figure()
ax = fig.add_subplot()
df = pd.read_csv('HVamp_full.csv', error_bad_lines=False, encoding = 'utf_8')
xdata = df['frequency']*1e3
ydata = df['dB']
sy = df['D']
ax.errorbar(df['frequency'], df['dB'], yerr=sy, fmt='.')

def H(f, L, R):
    s = 0 + 1j*2*np.pi*f
    numer = 4.02424*(1+(47*R*s)/10e9)
    denom = 1 + (47*s*(R+L*s))/10e9 + s*((100+L*s)*(1+(47*R*s)/10e9)+L*s*(1+(47*s*(100+R+L*s))/10e9))/1e9
    return 20*np.log10(np.abs(numer/denom))

#### fit data
# popt, pcov = curve_fit(H, xdata, ydata, p0=[0.5e-6,1], bounds=([1e-9,0],[inf,inf]), maxfev = 10000)
# print(popt, '\n', pcov)

# Hf_db = H(xdata, *popt)
# plt.plot(df['frequency'], Hf_db, 'k--')
# chi = sum((Hf_db - ydata)**2/sy**2)
# print(chi2.sf(chi,len(ydata)-len(popt)))

#### Plot data from SPICE
# with open('HVamp.txt', 'rb') as f:
#     data = np.array([str(l).split('\\t') for l in f.readlines()[1:]])
#     freq = np.array([float(d[0][2:]) for d in data])/1000
#     mag_out = [float(d[1].split(',')[0][1:-2]) for d in data]
# #     mag_in = [float(d[2].split(',')[0][1:-2]) for d in data]
# # mag = np.array(mag_out) + np.array(mag_in)
# mag = np.array(mag_out)
# ax.plot(freq, mag, label='SPICE simulation')

for L in np.linspace(1e-8, 100e-8, 21):
    plt.plot(df['frequency'], H(xdata, L, 0), ls='-', marker='', label=L*1e6)
    
ax.set_ylabel('dB')
ax.set_xlabel('kHz')
ax.set_xscale("log")
plt.legend()



In [ ]:
#### Fitting a linear model for the EMI filter
import pandas as pd
from scipy.stats.distributions import chi2
from numpy import pi

fig = plt.figure()
ax = fig.add_subplot()
df = pd.read_csv('EMIfilter.csv', error_bad_lines=False, encoding = 'utf_8')
xdata = df['frequency']*1e3
ydata = df['dB']
sy = df['D']
ax.errorbar(df['frequency'], df['dB'], yerr=sy, fmt='.')

def H2(f, L1, L2, R):
    ''' 50 Ohm source impedance '''
    numer = 1e18*(-5e9j + 47*pi*f*R)
    denom = -5e27j + pi*f*(94*(pi*f)**4*L1*L2 - 2e8*(pi*f)**2*(L1+L2)*(2375+47*R) + 1e18*(2400+47*R) + 1e10j*pi*f*(1e8*(95*L1+L2)+47*R) - 47j*(pi*f)**3*(4e8*L1*L2+(L1+L2)*R))
    return 20*np.log10(np.abs(numer/denom))

def H3(f, L1, L2, R):
    ''' 0 Ohm source impedance '''
    numer = 5e9*(5e9 + 47j*pi*f*R)
    denom = 25e18 + (pi*f)*(-5e9*(pi*f)*(95*L1+L2) + 94*(pi*f)**3*L1*L2 - 47j*(-5e9+(pi*f)**2*(L1+L2))*R)
    return 20*np.log10(np.abs(numer/denom))

def H4(f, L, R):
    ''' 50 Ohm source impedance 
    symmetric T-filter '''
    L1=L
    L2=L
    numer = 1e18*(-5e9j + 47*pi*f*R)
    denom = -5e27j + pi*f*(94*(pi*f)**4*L1*L2 - 2e8*(pi*f)**2*(L1+L2)*(2375+47*R) + 1e18*(2400+47*R) + 1e10j*pi*f*(1e8*(95*L1+L2)+47*R) - 47j*(pi*f)**3*(4e8*L1*L2+(L1+L2)*R))
    return 20*np.log10(np.abs(numer/denom))

def H5(f, L, R):
    ''' 0 Ohm source impedance 
    symmetric T-filter '''
    L1=L
    L2=L
    numer = 5e9*(5e9 + 47j*pi*f*R)
    denom = 25e18 + (pi*f)*(-5e9*(pi*f)*(95*L1+L2) + 94*(pi*f)**3*L1*L2 - 47j*(-5e9+(pi*f)**2*(L1+L2))*R)
    return 20*np.log10(np.abs(numer/denom))
    
fit_func = H5
popt, pcov = curve_fit(fit_func, xdata, ydata, p0=[1e-6,10], bounds=([0,0],[inf,inf]), maxfev = 10000)
print(popt, '\n', pcov)

Hf_db = fit_func(xdata, *popt)
plt.plot(df['frequency'], Hf_db, 'k--')
chi = sum((Hf_db - ydata)**2/sy**2)
print(chi2.sf(chi,len(ydata)-len(popt)))

ax.set_ylabel('dB')
ax.set_xlabel('kHz')
ax.set_xscale("log")


In [ ]:
import pandas as pd

plt.style.use('default')
df = pd.read_csv('HVAmp_scope.csv', error_bad_lines=False, encoding = 'utf_8')

mk = ['.', 's', 'x']
for i, d in enumerate(df.groupby(by='input')):
    plt.semilogx(d[1]['frequency'], d[1]['dB'], marker=mk[i], label='scope data ' + str(d[0]) + ' Vinput', ls='')

with open('HVamp.txt', 'rb') as f:
    data = np.array([str(l).split('\\t') for l in f.readlines()[1:]])
    freq = np.array([float(d[0][2:]) for d in data])/1000
    mag = [float(d[1].split(',')[0][1:-2]) for d in data]

plt.semilogx(freq, mag, label='SPICE simulation')
plt.ylabel('dB')
plt.xlabel('kHz')
plt.legend()


In [ ]:
import pandas as pd

plt.style.use('default')
df = pd.read_csv('EMIfilter.csv', error_bad_lines=False, encoding = 'utf_8')
plt.semilogx(df['frequency'], df['dB'], '.')

with open('EMIfilter.txt', 'rb') as f:
    data = np.array([str(l).split('\\t') for l in f.readlines()[1:]])
    freq = np.array([float(d[0][2:]) for d in data])/1000
    mag = [float(d[1].split(',')[0][1:-2]) for d in data]

plt.semilogx(freq, mag, label='SPICE simulation')
plt.ylabel('dB')
plt.xlabel('kHz')
plt.legend()

## Spectroscopy Data Analysis

In [ ]:
#### Load spectroscopy data
data = np.load('MB_1200kHz_split_186um_notilt_spectra.npy', allow_pickle=True).tolist()
# data = np.load('MB_1200kHz_split_186um_notilt_spectra_near_split.npy', allow_pickle=True).tolist()
# data = np.load('MB_1200kHz_split_186um_grand_canyon.npy', allow_pickle=True).tolist()

# plt.style.use('dark_background')
nice_fs = 22    #Nice font size
nice_lw = 4     #Nice linewidth
fig = plt.figure('spectra', figsize=(15,20))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel('729 AOM Frequency [MHz]', size=nice_fs*1)
ax.set_ylabel('Bright probability (with offset)', size=nice_fs*1)
ax.tick_params(axis='both', which='major', labelsize=nice_fs*0.7)
ax.get_xaxis().get_major_formatter().set_useOffset(False)
for i, (k, v) in enumerate(data.items()):
    xdata = v[0]
    ydata = v[1]+ 0.3*i
    ax.plot(xdata/1.0E6, ydata, color=colors[i], label=k)

    
center_freq = 178.512
ax_freq_com = 1.27
ax_freq_str = np.sqrt(3) * ax_freq_com
ax.axvline(center_freq - ax_freq_com/2, color='r', alpha=0.5, linewidth = 2*nice_lw)
ax.axvline(center_freq + ax_freq_com/2, color='r', alpha=0.5, linewidth = 2*nice_lw)
ax.axvline(center_freq - ax_freq_str/2, color='g', alpha=0.5, linewidth = 2*nice_lw)
ax.axvline(center_freq + ax_freq_str/2, color='g', alpha=0.5, linewidth = 2*nice_lw)
    
ax.legend(loc=4)
ax.grid()



In [ ]:
d = defaultdict(list)
for f in allvoltages:
    [d[k].append(v) for k, v in f.items()]

d['S11'] = list(np.array(d['S11']) - 0)
d['N9'] = list(np.array(d['N9']) - 0)
d['S12'] = list(np.array(d['S12']) + 0.01)
d['N8'] = list(np.array(d['N8']) - 0)
d['S13'] = list(np.array(d['S13']) - 0)

zshim = {'S21': 0.0, 'N1': 0.0, 'S20': 0.0, 'N2': 0.0, 'S19': 0.0, 'N3': 0.0, 'S14': 0.0, 'N4': 0.0, 'S13': 0.4484057326527151, 'N8': -0.13554507789550316, 'S12': 0.06742756883327992, 'N9': -0.13561294997392126, 'S11': 0.44870002610895593, 'N10': 0.0, 'S10': 0.0, 'N11': 0.0, 'S9': 0.0, 'N12': 0.0, 'S8': 0.0, 'N13': 0.0, 'S4': 0.0, 'N14': 0.0, 'S3': 0.0, 'N19': 0.0, 'S2': 0.0, 'N20': 0.0, 'S1': 0.0, 'N21': 0.0, 'S25': 0.0, 'S24': 0.0, 'S23': 0.0, 'S22': 0.0}
new_d = {}
n = 1
for k, v in zshim.items():
    new_d[k] = list(np.array(d[k]) + n*v)

endITVG = endsim - 0e-6 # we can't find the potential for voltages that don't exist! hence the minus 0.1us
volt_gen = vg
N_final = len(d['S21'])
t_exp, dx = np.linspace(0, (endITVG), N_final, retstep=True)
# t_exp = np.unique(sorted(np.concatenate((t_all, t_exp))))
print(len(t_exp), 'calls per iteration \t', endITVG, 'us')

bigarray = np.array([d[k] for k in electrode_ordering]).T
el_config = []
for row in bigarray:
    el_configi = [[[i+1], v] for i, v in enumerate(row)]
    el_configi.extend([[[el], 1] for el in volt_gen.rf_electrodes])
    el_config.append(el_configi)
    

In [ ]:
#### Single grid simulation
R = [[0.0, 0, 51]]
y_span_um = (155, 215)
n_y_points = int(1e-6*(y_span_um[1]-y_span_um[0])/1000e-9)+1 # set dy (space resolution)

y_points, dy = np.linspace(y_span_um[0], y_span_um[1], n_y_points, retstep=True)
y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config])

t_exp = np.linspace(0, 1e-6, 2)
y_poti = np.array([y_pot[0]]*len(t_exp))
print(index, el_config[index])

Nion = 2
q_ions = ct.e*   np.full(Nion, 1)
m = amu*                np.array([40, 40])
x0dot = um*             np.full(Nion, 0)

ymin = y_points[np.argmin(y_poti, axis=1)[0]]
q0 = 186.5
x0guess = um*           np.array([ymin, 2*q0 - ymin])
#     print(x0guess)
x0 = find_ic2(InterpolatedUnivariateSpline(y_points*1e-6, y_poti[0], k=3), Nion, x0guess, disp=False)
x0 *= np.array([1.00015, 0.99999])
#     print(x0*1e6)

heat_start = 5e-6
heat_end = 45e-6
t_eval = np.linspace(heat_start, heat_end, 4096)
outsol, ion_eval, pot_grid = heat_sim2(t_eval[-1], t_eval, y_poti, showplot=False, ms=1e-8)
plt.figure('potential')
plt.plot(y_points, y_poti[0]-np.min(y_poti[0]), marker='.')

N = 2**16
dx = t_eval[1]-t_eval[0]
ion_w = [np.fft.rfftfreq(N, dx)]*Nion
ion_fft = [np.fft.rfft(ion_eval[i]-np.average(ion_eval[i]), N) for i in range(Nion)]  

plt.figure('freq domain sim2')
ion_fft = np.abs(ion_fft)
plt.plot(ion_w[0], ion_fft[0]/np.max(ion_fft[0]), color=colors[0])
plt.plot(ion_w[1], ion_fft[1]/np.max(ion_fft[1]), color=colors[0])

In [ ]:
n_scan, dn = np.linspace(-10, 10, 21, retstep=True)

In [ ]:
#### Grid Simulation AT a Single Point in Time
indices = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,72,74,76,78,80,82]
# indices = [65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87]
# Rvec = [53,53,51.8,51.1,51.05,51.2,51.4,51.6,51.9,52.3,52.6,52.7,52.9,52.8,52.5,52.3,52.2,51.9,52.0,51.5,51.5]
# Rvec = [52.8,52.8,52.7,52.7,52.6,52.6,52.5,52.4,52.3,52.3,52.1,52.0,52,52,52,52,52,52,52,52,52,52,52]
nvec= [-0.5,-1.5,-2,-5,-11,-18,-30,-43.5,-56,-64,-68,-63,-57,-47,-37,-33,-29,-26,-24,-14,-13]
index = 15

v_effective = []
# for i, index in enumerate(indices):
# for i, r in enumerate(np.linspace(51, 53, 21)):
for i, n in enumerate(n_scan):
#     R = [[0.0, 0, Rvec[i]]]
#     R = [[0.0, 0, r]]
    zshim = {'S21': 0.0, 'N1': 0.0, 'S20': 0.0, 'N2': 0.0, 'S19': 0.0, 'N3': 0.0, 'S14': 0.0, 'N4': 0.0, 'S13': 0.4484057326527151, 'N8': -0.13554507789550316, 'S12': 0.06742756883327992, 'N9': -0.13561294997392126, 'S11': 0.44870002610895593, 'N10': 0.0, 'S10': 0.0, 'N11': 0.0, 'S9': 0.0, 'N12': 0.0, 'S8': 0.0, 'N13': 0.0, 'S4': 0.0, 'N14': 0.0, 'S3': 0.0, 'N19': 0.0, 'S2': 0.0, 'N20': 0.0, 'S1': 0.0, 'N21': 0.0, 'S25': 0.0, 'S24': 0.0, 'S23': 0.0, 'S22': 0.0}
    new_d = {}
    
    for k, v in zshim.items():
        new_d[k] = list(np.array(d[k]) + n*v)
        
    endITVG = 5e-6 # we can't find the potential for voltages that don't exist! hence the minus 0.1us
    volt_gen = vg
    N_final = len(d['S21'])
    t_exp, dx = np.linspace(0, (endITVG), N_final, retstep=True)
#     print(len(t_exp), 'calls per iteration \t', endITVG, 'us')

    bigarray = np.array([new_d[k] for k in electrode_ordering]).T
    el_config = []
    for row in bigarray:
        el_configi = [[[i+1], v] for i, v in enumerate(row)]
        el_configi.extend([[[el], 1] for el in volt_gen.rf_electrodes])
        el_config.append(el_configi)
    
    R = [[0.0, 0.0, 48]]
    y_span_um = (140, 230)
    n_y_points = int(1e-6*(y_span_um[1]-y_span_um[0])/1000e-9)+1 # set dy (space resolution)

    y_points, dy = np.linspace(y_span_um[0], y_span_um[1], n_y_points, retstep=True)
    y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
    y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config])
    xaxis = y_points
    yaxis = y_pot
    adaptive_yaxis = True
    q_min = [y_points[np.where(np.array(y) == min(np.array(y)))][0] for y in y_pot]

    ####
    t_exp = np.linspace(0, endITVG, 50)
    y_poti = np.array([y_pot[index]]*len(t_exp))
    print(index, el_config[index])
    v_effective.append(el_config[index])

    Nion = 2
    q_ions = ct.e*   np.full(Nion, 1)
    m = amu*                np.array([40, 40])
    x0dot = um*             np.full(Nion, 0)

    ymin = y_points[np.argmin(y_poti, axis=1)[0]]
    q0 = 186.5
    x0guess = um*           np.array([ymin, 2*q0 - ymin])
#     print(x0guess)
    x0 = find_ic2(InterpolatedUnivariateSpline(y_points*1e-6, y_poti[0], k=3), Nion, x0guess, disp=False)
    x0 *= np.array([1.00015, 0.99999])
#     print(x0*1e6)

    heat_start = 5e-6
    heat_end = 45e-6
    t_eval = np.linspace(heat_start, heat_end, 4096)
    outsol, ion_eval, pot_grid = heat_sim2(t_eval[-1], t_eval, y_poti, showplot=False, ms=1e-8)
#     plt.figure('potential')
#     plt.plot(y_points, y_poti[0]-np.min(y_poti[0])+i*0.75, marker='.', color=colors[i])

    N = 2**16
    dx = t_eval[1]-t_eval[0]
    ion_w = [np.fft.rfftfreq(N, dx)]*Nion
    ion_fft = [np.fft.rfft(ion_eval[i]-np.average(ion_eval[i]), N) for i in range(Nion)]  

    plt.figure('freq domain sim2')
    ion_fft = np.abs(ion_fft)
    plt.plot(ion_w[0], dn*(ion_fft[0]/np.max(ion_fft[0]))+n, color=colors[i])
    plt.plot(ion_w[1], dn*(ion_fft[1]/np.max(ion_fft[1]))+n, color=colors[i])
    
plt.xlabel('Frequency (Hz)')
plt.ylabel('Applied Electric Field (V/cm)')
plt.title('Simulated Spectroscopy Across Ez Fields at Index 15')


In [ ]:
#### overlay MB_1200kHz_split_186um_notilt_spectra.npy
indices = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,72,74,76,78,80,82]
spectroscopy_data = {1:(1.307, 2.266),
                    6:(1.307, 2.253),
                    11:(1.255, 2.173),
                    16:(1.145, 1.959),
                    21:(0.976, 1.656),
                    26:(0.826, 1.347),
                    31:(0.845, 1.27),
                    36:(0.967, 1.61),
                    41:(1.229, 2.115),
                    46:(1.459, 2.528),
                    51:(1.605, 2.788),
                    56:(1.573, 2.745),
                    61:(1.513, 2.639),
                    66:(1.328, 2.327),
                    71:(1.051, 1.82),
                    73:(0.925, 1.586),
                    75:(0.801, 1.343),
                    77:(0.695, 1.195),
                    79:(0.516, 1.119),
                    81:(1.205, 1.409),
                    83:(1.24, 1.24)}

com = []
stretch = []
for i, (k, v) in enumerate(spectroscopy_data.items()):
    com.append(v[0])
    stretch.append(v[1])
    
# for i, index in enumerate(indices):
# for i, r in enumerate(np.linspace(51, 53, 21)):
# for i, n in enumerate(n_scan):
#     v = spectroscopy_data[index+1]
#     plt.vlines(v[0]*1e6, n, n+dn, colors=colors[i], linewidth=5)
#     plt.vlines(v[1]*1e6, n, n+dn, colors=colors[i], linewidth=5)
    
fig, ax = plt.subplots()
plt.plot(list(spectroscopy_data.keys()), com, color='k', marker='.')
plt.plot(list(spectroscopy_data.keys()), stretch, color = 'b', marker='.')
# plt.plot(list(spectroscopy_data.keys()), np.array(stretch)/np.array(com))
# plt.hlines(np.sqrt(3), list(spectroscopy_data.keys())[0], list(spectroscopy_data.keys())[-1])
plt.xlabel('Step')
plt.ylabel('Axial frequency (MHz)')

In [ ]:
from matplotlib.lines import Line2D


custom_lines = [Line2D([0], [0], color='k'), Line2D([0], [0], color='b')]
custom_lines2 = [Line2D([0], [0], color='tab:grey', ls='--', lw=2), Line2D([0], [0], color='tab:grey', ls='solid', marker='.', lw=2)]

ax.legend(custom_lines, ['COM mode', 'Stretch mode'])
ax2 = ax.twinx()
ax2.legend(custom_lines2, ['Simulated', 'Measured'], loc=(0.815, 0.68))


In [ ]:
plt.title('Simulated and Measured Axial Frequencies During Ion Splitting')

In [ ]:
#### Searching for best z_shim and z_pos
z_shim, dz_shim = np.linspace(30, 0, 21, retstep=True)
z_pos, dz_pos = np.linspace(51, 52.5, 21, retstep=True)
z_shim=[13.5]
z_pos=[51.675]

zshim = {'S21': 0.0, 'N1': 0.0, 'S20': 0.0, 'N2': 0.0, 'S19': 0.0, 'N3': 0.0, 'S14': 0.0, 'N4': 0.0, 'S13': 0.4484057326527151, 'N8': -0.13554507789550316, 'S12': 0.06742756883327992, 'N9': -0.13561294997392126, 'S11': 0.44870002610895593, 'N10': 0.0, 'S10': 0.0, 'N11': 0.0, 'S9': 0.0, 'N12': 0.0, 'S8': 0.0, 'N13': 0.0, 'S4': 0.0, 'N14': 0.0, 'S3': 0.0, 'N19': 0.0, 'S2': 0.0, 'N20': 0.0, 'S1': 0.0, 'N21': 0.0, 'S25': 0.0, 'S24': 0.0, 'S23': 0.0, 'S22': 0.0}
v_effective = []
# cost = []
for z_shimi in z_shim:
    cost_row = []
    for z_posi in z_pos:
        print(z_shimi, z_posi)
        new_d = {}
        for k, v in zshim.items():
            new_d[k] = list(np.array(d[k]) + z_shimi*v)

        bigarray = np.array([new_d[k] for k in electrode_ordering]).T
        el_config = []
        for row in bigarray:
            el_configi = [[[i+1], v] for i, v in enumerate(row)]
            el_configi.extend([[[el], 1] for el in volt_gen.rf_electrodes])
            el_config.append(el_configi)

        R = [[0.0, 0.0, z_posi]]
        y_span_um = (140, 230)
        n_y_points = int(1e-6*(y_span_um[1]-y_span_um[0])/1000e-9)+1 # set dy (space resolution)

        y_points, dy = np.linspace(y_span_um[0], y_span_um[1], n_y_points, retstep=True)
        y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
        y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config[::down]])

        ####
        costi = 0
        for index in [20, 60]:
            t_exp = np.linspace(0, 5e-6, 50)
            y_poti = np.array([y_pot[index]]*len(t_exp))
#             print(index, el_config[index])
            v_effective.append(el_config[index])

            Nion = 2
            q_ions = ct.e*   np.full(Nion, 1)
            m = amu*                np.array([40, 40])
            x0dot = um*             np.full(Nion, 0)

            ymin = y_points[np.argmin(y_poti, axis=1)[0]]
            x0guess = um*           np.array([ymin, 2*q0 - ymin])
        #     print(x0guess)
            x0 = find_ic2(InterpolatedUnivariateSpline(y_points*1e-6, y_poti[0], k=3), Nion, x0guess, disp=False)
            x0 *= np.array([1.00015, 0.99999])
        #     print(x0*1e6)

            heat_start = 5e-6
            heat_end = 45e-6
            t_eval = np.linspace(heat_start, heat_end, 4096)
            outsol, ion_eval, pot_grid = heat_sim2(t_eval[-1], t_eval, y_poti, showplot=False, ms=1e-8)
            plt.figure('potential')
            plt.plot(y_points, y_poti[0], marker='.', color=colors[i])

            N = 2**16
            dx = t_eval[1]-t_eval[0]
            ion_w = [np.fft.rfftfreq(N, dx)]*Nion
            ion_fft = [np.abs(np.fft.rfft(ion_eval[i]-np.average(ion_eval[i]), N)) for i in range(Nion)]  
            for ffti in ion_fft:
                ffti = ffti/np.max(ffti)
                peak_indices = find_peaks(ffti, 0.075, distance=160)[0][:2]
                peak_w = ion_w[0][peak_indices]
            
            for i, wi in enumerate(peak_w):
                costi += ((spectroscopy_data[index+1][i]*1e6 - wi)/(spectroscopy_data[index+1][i]*1e6))**2

            plt.figure('freq domain sim2')
            ion_fft = np.abs(ion_fft)
            plt.plot(ion_w[0], ion_fft[0]/np.max(ion_fft[0])+i*0.75)
            plt.plot(ion_w[1], ion_fft[1]/np.max(ion_fft[1])+i*0.75)
#         print("cost: ", costi)
#         cost_row.append(costi)
#     cost.append(cost_row)
            

In [ ]:
extent = [z_pos[0], z_pos[-1]+dz_pos, z_shim[0], z_shim[-1]+dz_shim]

# Plot heatmap
plt.imshow(
    np.log(np.array(cost)),
    extent=extent,
    #vmin = 0.0,
    #vmax = 5.0,
    aspect = 'auto', # Prevent the image from getting stretched out
    interpolation = 'nearest', # ???
    origin = 'lower', # Important to do this so that the first row goes on the bottom
    cmap = 'viridis',
)
plt.xlabel('z-position (um)')
plt.ylabel('z-shim (V/cm)')

In [ ]:
v_eff = np.array([[vi[8][-1],vi[9][-1],vi[10][-1],vi[11][-1],vi[12][-1]] for vi in v_effective]).T
for v in v_eff:
    plt.plot(indices, v, marker='.')
    
plt.ylabel('effective voltage (V)')
plt.xlabel('index')

In [ ]:
#### overlay MB_1200kHz_split_186um_notilt_spectra_near_split.npy
spectroscopy_data = {66:(1.347, 2.347),
                    67:(1.303, 2.273),
                    68:(1.25, 2.187),
                    69:(1.2, 2.091),
                    70:(1.141, 1.989),
                    71:(1.088, 1.877),
                    72:(1.021, 1.766),
                    73:(0.963, 1.653),
                    74:(0.896, 1.531),
                    75:(0.838, 1.413),
                    76:(0.781, 1.301),
                    77:(0.723, 1.181),
                    78:(0.666, 1.085),
                    79:(0.613, 1.016),
                    80:(0.681, 0.909),
                    81:(0.74, 0.951),
                    82:(0.855, 1.01),
                    83:(0.965, 1.083),
                    84:(1.072, 1.156),
                    85:(1.195, 1.195),
                    86:(1.264, 1.264),
                    87:(1.325, 1.325),
                    88:(1.372, 1.372)}

com = []
stretch = []
for i, (k, v) in enumerate(spectroscopy_data.items()):
    com.append(v[0])
    stretch.append(v[1])
    
# for i, index in enumerate(indices):
#     v = spectroscopy_data[index+1]
#     plt.vlines(v[0]*1e6, i*0.75, i*0.75+1, colors=colors[i], linewidth=5)
#     plt.vlines(v[1]*1e6, i*0.75, i*0.75+1, colors=colors[i], linewidth=5)
    
plt.plot(list(spectroscopy_data.keys()), com)
plt.plot(list(spectroscopy_data.keys()), stretch)
# plt.plot(list(spectroscopy_data.keys()), np.array(stretch)/np.array(com))
# plt.hlines(np.sqrt(3), list(spectroscopy_data.keys())[0], list(spectroscopy_data.keys())[-1])
plt.xlabel('Step')
plt.ylabel('Axial frequency (MHz)')

In [ ]:
#### Coefficient Simulation AT a Single Point in Time
indices = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,72,74,76,78,80,82]
# indices = [20]
for i, index in enumerate(indices):
    t_exp = np.linspace(0, 5e-6, 50)
    q_expi = np.repeat(q_exp[index], len(t_exp))
    a_expi = np.repeat(a_exp[index], len(t_exp))
    d_expi = np.repeat(d_exp[index], len(t_exp))
    y_expi = np.repeat(y_exp[index], len(t_exp))
    c_expi = np.repeat(c_exp[index], len(t_exp))
    
    Nion = 2
    q_ions = elem_charge*   np.full(Nion, 1)
    m = amu*                np.array([40, 40])
    x0dot = um*             np.full(Nion, 0)
    x0guess = um*           np.array([q_exp[0]-1, q_exp[0]+1])

    y_poti = [y_expi[0]*(y_points-q_expi[0]) + a_expi[0]*(y_points-q_expi[0])**2 + c_expi[0]*(y_points-q_expi[0])**3 + d_expi[0]*(y_points-q_expi[0])**4]
    ymin = y_points[np.argmin(y_poti, axis=1)[0]]
    x0guess = um*           np.array([ymin, 2*q0 - ymin])
    print(x0guess)
    x0 = find_ic2(InterpolatedUnivariateSpline(y_points*1e-6, y_poti[0], k=3), Nion, x0guess, disp=False)
    x0 *= np.array([1.00001, 0.999999])
    print(x0*1e6)
    
    heat_start = 5e-6
    heat_end = 45e-6

    t_eval = np.linspace(heat_start, heat_end, 4096)
    outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_expi, t_eval, showplot=False, a=a_expi, b=d_expi, g=y_expi, c=c_expi, deg=1, ms=1e-8)
    print('heat gained (quanta): ', n)

    N = 2**16
    dx = t_eval[1]-t_eval[0]
    ion_w = [np.fft.rfftfreq(N, dx)]*Nion
    ion_fft = [np.fft.rfft(y_eval[i]-np.average(y_eval[i]), N) for i in range(Nion)]  

    plt.figure('freq domain')
    ion_fft = np.abs(ion_fft)
    plt.plot(ion_w[0], ion_fft[0]/np.max(ion_fft[0])+i*0.75, color=colors[i])
    plt.plot(ion_w[1], ion_fft[1]/np.max(ion_fft[1])+i*0.75, color=colors[i])


In [ ]:
#### Compares compute_potential with compute_total_potential_axes
indices = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,72,74,76,78,80,82]
for i, index in enumerate(indices):
    plt.plot(y_points, y_pot[index], ls='--',  color=colors[i])
    plt.plot(y_points, y_exp[index]*(y_points-186.5) + c_exp[index]*(y_points-186.5)**3 + a_exp[index]*(y_points-186.5)**2 + d_exp[index]*(y_points-186.5)**4 + np.repeat(const_exp[index], len(y_points)),  color=colors[i])
    plt.xlabel('um')
    plt.ylabel('V')
#     plt.legend()

indices = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,72,74,76,78,80,82]
for i, index in enumerate(indices[:5]):
    t_exp = np.linspace(0, 5e-6, 50)
    y_pot0 = [y_exp[index]*(y_points-q_exp[index]) + a_exp[index]*(y_points-q_exp[index])**2 + c_exp[index]*(y_points-q_exp[index])**3 + d_exp[index]*(y_points-q_exp[index])**4]
    
    plt.plot(y_points, y_pot0[0], color=colors[i])
    plt.plot(y_points, y_pot[index]-y_pot[index][int(len(y_points)/2)], color=colors[i], ls='--')

In [ ]:
#### Coefficient Simulation Using Taylor Fitted Coefficients
indices = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,72,74,76,78,80,82]
indices = [20]
for i, index in enumerate(indices):
    t_exp = np.linspace(0, 5e-6, 50)
    
    ## Fit the potential with a quartic polynomial
    def f6(y, a, b, c, d, y0, g):
        return a*(y-y0)**2 + b + c*(y-y0)**3 + d*(y-y0)**4 + g*(y-y0)
    
    y = np.array(y_pot[index])
    minindex = np.where(y_points == 186)
    interval = [int(minindex-20/dy), int(minindex+20/dy)]
    xdata = y_points[interval[0]:interval[1]]
    ydata = y[interval[0]:interval[1]]
    popt, pcov = curve_fit(f6, xdata, ydata, p0=[a_exp[index], const_exp[index], c_exp[index], d_exp[index], y_points[minindex][0], y_exp[index]], maxfev=10000)
    print(popt)
    
    q_expi = np.repeat(popt[4], len(t_exp))
    a_expi = np.repeat(popt[0], len(t_exp))
    d_expi = np.repeat(popt[3], len(t_exp))
    y_expi = np.repeat(popt[5], len(t_exp))
    c_expi = np.repeat(popt[2], len(t_exp))
    const_expi = np.repeat(popt[1], len(t_exp))
    
    y_pot_forward = [const_exp[index] + y_exp[index]*(y_points-q_exp[index]) + a_exp[index]*(y_points-q_exp[index])**2 + c_exp[index]*(y_points-q_exp[index])**3 + d_exp[index]*(y_points-q_exp[index])**4]    
    y_poti = [popt[1] + popt[5]*(y_points-popt[4]) + popt[0]*(y_points-popt[4])**2 + popt[2]*(y_points-popt[4])**3 + popt[3]*(y_points-popt[4])**4]
    
    ## Compare the different potentials
#     plt.plot(y_points, y_pot_forward[0], label='Forward ITVG')
#     plt.plot(y_points, y_pot[index], label='Full Potential', marker='.')
#     plt.plot(y_points, y_poti[0], label='Taylor Fit on Full Potential')
#     plt.legend()
    
    ## Simulate ions
    Nion = 2
    q_ions = elem_charge*   np.full(Nion, 1)
    m = amu*                np.array([40, 40])
    x0dot = um*             np.full(Nion, 0)
    x0guess = um*           np.array([q_exp[0]-1, q_exp[0]+1])
    
    ymin = y_points[np.argmin(y_poti, axis=1)[0]]
    x0guess = um*           np.array([ymin, 2*q0 - ymin])
    print(x0guess)
    x0 = find_ic2(InterpolatedUnivariateSpline(y_points*1e-6, y_poti[0], k=3), Nion, x0guess, disp=False)
    x0 *= np.array([1.00001, 0.999999])
    print(x0*1e6)
    
    heat_start = 5e-6
    heat_end = 45e-6

    t_eval = np.linspace(heat_start, heat_end, 4096)
    outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_expi, t_eval, showplot=False, a=a_expi, b=d_expi, g=y_expi, c=c_expi, deg=1, ms=1e-8)
    print('heat gained (quanta): ', n)

    N = 2**16
    dx = t_eval[1]-t_eval[0]
    ion_w = [np.fft.rfftfreq(N, dx)]*Nion
    ion_fft = [np.fft.rfft(y_eval[i]-np.average(y_eval[i]), N) for i in range(Nion)]  

    plt.figure('freq domain')
    ion_fft = np.abs(ion_fft)
    plt.plot(ion_w[0], ion_fft[0]/np.max(ion_fft[0])+i*0.75, color=colors[i])
    plt.plot(ion_w[1], ion_fft[1]/np.max(ion_fft[1])+i*0.75, color=colors[i])


In [ ]:
#### Grid Simulation UP TO a Point in Time
indices = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,72,74,76,78,80,82]
# indices = [20]
for i, index in enumerate(indices):
    dt = 1e-6
    t_exp = np.linspace(0, index+20, index+20)*dt
    y_poti = np.vstack((y_pot[:index], [y_pot[index]]*20))

    Nion = 2
    q_ions = ct.e*   np.full(Nion, 1)
    m = amu*                np.array([40, 40])
    x0dot = um*             np.full(Nion, 0)

    ymin = y_points[np.argmin(y_poti, axis=1)[0]]
    x0guess = um*           np.array([ymin, 2*q0 - ymin])
    print(x0guess)
    x0 = find_ic2(InterpolatedUnivariateSpline(y_points*1e-6, y_poti[0], k=3), Nion, x0guess, disp=False)
    print(x0*1e6)

    heat_start = (index+5)*dt
    heat_end = t_exp[-1]+5e-6
    t_eval = np.linspace(heat_start, heat_end, 4096)
    outsol, ion_eval, pot_grid = heat_sim2(t_eval[-1], t_eval, y_poti, showplot=False, ms=1e-8)
#     plt.figure('potential')
#     plt.plot(y_points, y_poti[0])

    N = 2**16
    dx = t_eval[1]-t_eval[0]
    ion_w = [np.fft.rfftfreq(N, dx)]*Nion
    ion_fft = [np.fft.rfft(ion_eval[i]-np.average(ion_eval[i]), N) for i in range(Nion)]

    plt.figure('freq domain')
    ion_fft = np.abs(ion_fft)
    plt.plot(ion_w[0], ion_fft[0]/np.max(ion_fft[0])+i, color=colors[i])
    plt.plot(ion_w[1], ion_fft[1]/np.max(ion_fft[1])+i, color=colors[i])


In [ ]:
#### View simulated split data
import pandas as pd
df = pd.read_csv('MB_split_simulated.csv', error_bad_lines=False, encoding = 'utf_8')
a_sim = df.iloc[:, 0]
b_sim = df.iloc[:, 1]
w_sim = df.iloc[:, 2]
w2_sim = df.iloc[:, 3]

d_sim = [np.roots([bi, 0, 2*ai, 0, 0, -ct.e/(2*pi*ct.epsilon_0)]) for ai, bi in zip(a_sim*1e12, b_sim*1e24)]
d_sim = np.real(np.concatenate([[di*1e6 for di in d if np.isreal(di) and np.real(di) > 0] for d in d_sim]))
# print(d_sim)
f_sim = np.sqrt(ct.e/(40*ct.m_u)*(2*a_sim*1e12 + 3*b_sim*1e24*(d_sim*1e-6)**2))/(2*pi)
ep_sim = ct.e**2/(pi*ct.epsilon_0*(40*ct.m_u)*(2*pi*f_sim)**2*(d_sim*1e-6)**3)
f2_sim = np.sqrt((2*pi*f_sim)**2*(1+ep_sim))/(2*pi)

# plt.plot(a_sim/np.max(a_sim))
# plt.plot(b_sim/np.max(b_sim))
plt.plot(f_sim[:100]/1e6, color='k', ls='--')
plt.plot(f2_sim[:100]/1e6, color='b', ls='--')

In [ ]:
import csv
with open('MaxBeta_split_simulated_051121.csv', mode='w') as mb:
    mb_writer = csv.writer(mb, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for a,b,f1,f2 in zip(a_sim, b_sim, f_sim, f2_sim):
        mb_writer.writerow([a, b, f1, f2])


## Waveforms

In [34]:
#### S13 waveform for transport 
labels = ['bang-bang dt=1us', 'predistorted dt=1us', 'predistorted dt=0.8us', 'predistorted dt=0.6us']
t_dat = np.load("predistorted_protocols_times_array_list.npy", allow_pickle=True)
v_dat = np.load("predistorted_protocols_volts_array_list.npy", allow_pickle=True)

plt.figure('measured waveform')
indices = [0]
for i in indices:
    plt.plot(t_dat[i]*1e6, v_dat[i], 'k', label=labels[i] + " (measured)")
    
plt.plot(t_dat[0]*1e6, v_exp['S13'].__call__(np.maximum(0, t_dat[0]-6e-6)), '--', label='SPICE simulation')

plt.legend()

In [51]:
#### S12 and S13 waveform for transport dt=392ns - 6/27/21
t_dat = np.load("MB_fasttransport_4MHz_dt=392ns_south_num_periods=1_times_array_list.npy", allow_pickle=True)
v_dat = np.load("MB_fasttransport_4MHz_dt=392ns_south_num_periods=1_volts_array_list.npy", allow_pickle=True)

labels = ['S12', 'S13']
col = ['b', 'g']

plt.figure('measured waveform')
indices = [0, 1]
for i in indices:
    plt.plot(t_dat[i]*1e6, v_dat[i], color=col[i], label=labels[i] + " (measured)")
    plt.plot(t_dat[0]*1e6, v_exp[labels[i]].__call__(np.maximum(0, t_dat[0]-16.8e-6)), '--', color=col[i], label=labels[i] + ' (simulated)')

plt.xlabel('time [us]')
plt.ylabel('voltage [V]')
plt.legend()

In [58]:
#### S12 and S13 waveform for transport dt=800ns - 6/27/21
t_dat = np.load("MB_fasttransport_4000kHz_dt=800ns_south_num_periods=2_times_array_list.npy", allow_pickle=True)
v_dat = np.load("MB_fasttransport_4000kHz_dt=800ns_south_num_periods=2_volts_array_list.npy", allow_pickle=True)

labels = ['S12', 'S13']
col = ['b', 'g']

plt.figure('measured waveform')
indices = [0, 1]
for i in indices:
    plt.plot(t_dat[i]*1e6, v_dat[i], color=col[i], label=labels[i] + " (measured)")
    plt.plot(t_dat[0]*1e6, v_exp[labels[i]].__call__(np.maximum(0, t_dat[0]-16.8e-6)), '--', color=col[i], label=labels[i] + ' (simulated)')

plt.xlabel('time [us]')
plt.ylabel('voltage [V]')
plt.legend()

## My Own Splitting Method (for Thesis)

In [ ]:
Tf = np.linspace(5, 60, 56)

split_heat = []
for T0 in Tf:
    eps0 = ct.epsilon_0
    K = ct.e/(4*pi*eps0)
    Ca = 40*ct.m_u
    Sr = 88*ct.m_u
    Ba = 133*ct.m_u
    mass = Ca
    
    f0 = 1.2e6*(Ca/mass)**0.5
    q0 = 0 # center of split [um]
    bmax = ((2*np.pi*f0)**2*mass/(2*ct.e)/(2*K)**0.4)**(5/3)
    d_CP = 0.5*(2*K/(bmax))**0.2 ## half the separation at CP

    t_ideal, dx = np.linspace(0, T0*1e-6, 4096, retstep=True) ## s
    a0 = (2*pi*f0)**2*mass/(2*ct.e)
    d0 = (K/a0)**(1/3)
    print(d0/2*1e6)
    d_ideal = np.array([qsta2(t, (t_ideal[0], t_ideal[-1]), qspan=(d0/2*1e6, d0/2*1e6*10), f=f0*T0) for t in t_ideal]) ## um (half the separation)

    t_CP = interp1d(d_ideal, t_ideal).__call__(d_CP*1e6)
    print('d_CP: ', d_CP*1e6, 't_CP: ', t_CP)

    ln_ideal = np.repeat(2*pi*f0/sqrt(ct.e), len(t_ideal))
    lp_ideal = np.sqrt(4*K/mass*(d_ideal*2e-6)**(-3) + ln_ideal**2)
    a_ideal = mass/8*(3*lp_ideal**2 - 5*ln_ideal**2) ## V/m^2
    b_ideal = 2*K/(d_ideal*2e-6)**5 - 2*a_ideal/(d_ideal*2e-6)**2 ## V/m^4

    w_ideal = np.array([(p_derivative(phi, point=[t, d, a, b, mass], order=2)*ct.e/mass)**0.5 for t, d, a, b in zip(t_ideal, d_ideal*2e-6, a_ideal, b_ideal)])
    w2_ideal = np.array([(ct.e/mass*(2*a + 3*b*d**2))**0.5 for a, b, d in zip(a_ideal, b_ideal, d_ideal*2e-6)])
    w2_stretch_ideal = w2_ideal*(1+4*ct.e*K/(mass*(w2_ideal)**2*(d_ideal*2e-6)**3))**0.5

    a_ideal *= 1e-12
    b_ideal *= 1e-24

    #### Simulate ions
    t_exp = t_ideal
    q_exp = np.full(len(t_ideal), q0)
    a_exp = a_ideal
    b_exp = b_ideal
#     plt.plot(t_exp, a_exp)
                    
    hbar = ct.hbar
    eps0 = ct.epsilon_0
    k0 = 1/(4*pi*eps0)
    amu = ct.m_u
    elem_charge = ct.e
    um = 1e-6

    Nion = 2
    q_ions = elem_charge*   np.full(Nion, 1)
    m =                 np.array([mass, mass])
    x0dot = um*             np.full(Nion, 0)
    x0guess = um*           np.array([q_exp[0]-1, q_exp[0]+1])

    x0 = find_ic(q_exp[0]/1e6, a_exp[0]*1e12, Nion, x0guess)
    print(x0*1e6)

    heat_start = t_exp[-1]
    heat_end = t_exp[-1] + 25e-6

    t_eval, dx = np.linspace(heat_start, heat_end, 4096, retstep=True)
    outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_exp, t_eval, showplot=False, a=a_exp, b=b_exp, g=None, c=None, deg=1, ms=1e-8)
    print('heat gained (quanta): ', n)
       
    N = 2**16
    ion_w = np.fft.rfftfreq(N, dx)
    ion_fft = np.fft.rfft((y_eval[0]-q_eval) - np.average((y_eval[0]-q_eval)), N)
    peaks, _ = find_peaks(np.abs(ion_fft), np.max(np.abs(ion_fft))/4)
    f_peak = ion_w[peaks[-1]]
    print('end freq: ', f_peak)
    window = gaussian(ion_w, f_peak, 0.1e6)

    n = get_heat(np.fft.irfft(ion_fft*window, N)[:len(t_eval)], 2*pi*f_peak, m_=mass)
    print('heat: ', n, '\n')
    split_heat.append(n)


In [ ]:
max(b_exp)

In [ ]:
print([s for s in split_heat])

In [ ]:
from matplotlib.lines import Line2D

h_T_Ca = [31030.349287188157, 12413.953044381038, 7298.630575924747, 3510.939305084948, 2017.6395305992164, 1188.1566023494618, 652.5621539153399, 432.28053075078225, 242.35202843560162, 164.710222772729, 100.02532657896555, 64.33398191630303, 44.54637186926699, 26.206335163365335, 19.902966473902673, 11.9039528227267, 8.488162222889649, 5.875432026113415, 3.6523823546285006, 2.8647580339933043, 1.6857211014753803, 1.3438614076825774, 0.8421910103461344, 0.5971852957461884, 0.4511561248284081, 0.2543939148988819, 0.2354175120713044, 0.12319685576662551, 0.10397938823383517, 0.07194474127947866, 0.04111442693840466, 0.03986220179835899, 0.017847800013275014, 0.018847638422114666, 0.009529827408798956, 0.0076113041460743385, 0.005814048837632633, 0.002126649217739157, 0.003559517839324575, 0.00071191941070115, 0.0013182726683338307, 0.000693682765060617, 0.00019371129132167628, 0.0005442366148113113, 8.916997524530875e-06, 0.00015765289030307756, 9.031575979334318e-05, 5.0392346928219295e-05, 7.807709050182959e-05, 1.7550782147612258e-05, 2.5127122311996352e-05, 0.00010561365974232202, 1.675559271791951e-06, 5.713281019054253e-05, 4.9977255181902484e-05, 4.5653680924709024e-05]
h_T_Sr = [156560.08543657127, 94623.46337404409, 59498.514347750024, 31096.53338858783, 17626.193784094085, 12451.92156721494, 8202.624973575392, 4857.822731281415, 3297.804812710546, 2452.8358048159953, 1629.3602513822314, 1068.3896768256272, 806.5552806631606, 595.6337476705219, 398.7223079396819, 290.2524449682287, 229.50636739461004, 163.3243769702086, 113.12434727032243, 90.53268267399145, 70.92516668345178, 48.690536797768566, 36.253779231012416, 30.62532161914686, 22.83795824618117, 15.687459613243076, 12.809242449059177, 10.70572801948134, 7.5239911955891685, 5.516751358940619, 4.8047677893873635, 3.7295560357392357, 2.5583736762843494, 2.117031087101105, 1.8244364137318207, 1.2713216526260405, 0.9259332702678801, 0.8583135487938812, 0.6713365702613325, 0.4298068936573552, 0.3660067216963548, 0.346966476751382, 0.23242075538241708, 0.15247805617273746, 0.15422302208555047, 0.1314432208591102, 0.07621122539652246, 0.06115932892012312, 0.06430208532000749, 0.04407513091519742, 0.025512634918352955, 0.027409395102505188, 0.024148013333332077, 0.012568925842176204, 0.010090997499812196, 0.01228265186084874]
h_T_Ba = [298921.21350067336, 139900.51177480214, 86659.55501854324, 62348.3354447187, 39794.96608509279, 23146.131366340385, 14680.471956629346, 10964.096505697587, 8149.805266103653, 5451.97226829091, 3597.038257490508, 2673.3194291018417, 2112.644675904647, 1557.3360975353637, 1079.7613936907442, 797.3426685278724, 643.1062353619161, 503.66646979636516, 365.33619643846686, 269.44909115768417, 218.88275744304866, 179.193764067663, 134.96070615262119, 98.94766933431559, 79.7031399279353, 67.94626203390715, 53.650498981946335, 39.201700842873834, 30.502333632781884, 26.500652324637876, 22.245359268286915, 16.647621104003203, 12.409004469088991, 10.55642708033582, 9.307481485713133, 7.334214158381726, 5.36814159645461, 4.365172066249209, 3.9254658907338342, 3.25962463991007, 2.4038174599407327, 1.8670593032161178, 1.6857074140808648, 1.468634746430704, 1.0990206742402395, 0.8089374157043434, 0.7210818625802311, 0.6713314953501047, 0.5218855200660304, 0.3604172074895525, 0.2993338512944708, 0.29818979420461533, 0.2546371963745708, 0.17181234603458206, 0.12431561966672774, 0.12416917878387784]
hSTA_Ca = [2840.095448046702, 783.3115536623667, 197.1308132894436, 43.74034633878774, 8.050216318866267, 1.046348598018772, 0.05410107276067466, 0.006831022912639495, 0.02242486389052651, 0.01930010196519903, 0.010765761136369926, 0.005163487849388397, 0.0022984295421921703, 0.001191267317844142, 0.0005992663605350542, 0.0004326209532734918, 0.00021607324468043908, 0.00019125197679992543, 7.876168539681115e-05, 8.828703613975048e-05, 3.004997469192702e-05, 4.483183425722448e-05, 1.2858744938534772e-05, 2.2517898849465254e-05, 6.834031816294669e-06, 1.1587811719928914e-05, 5.4602408881373455e-06, 5.9615945429194624e-06, 4.533759989070519e-06, 2.4953906297629417e-06, 4.602892182893598e-06, 1.112701477422366e-06, 4.200982359735643e-06, 5.546494783743594e-07, 3.6010951104685015e-06, 3.755725812082357e-07, 3.4413164230091212e-06, 5.207453112964182e-07, 2.7278510913988978e-06, 9.47769050816452e-07, 2.2551945357779364e-06, 1.1616711016004658e-06, 2.0855235122224918e-06, 1.354643967894689e-06, 1.5983747397987967e-06, 1.765444109198865e-06, 1.4095897175881486e-06, 1.7279317140575153e-06, 1.449785474492822e-06, 1.765224217455701e-06, 1.2321697129484758e-06, 1.9700285409827212e-06, 1.2550039212204632e-06, 1.7372074875928125e-06, 1.4454579063144976e-06, 1.7113626865339463e-06]
hSTA_Sr = [29432.624483748285, 13353.259078516283, 5951.824454522821, 2567.514927012301, 1063.1987401310357, 421.8409577762273, 159.03457039111365, 56.17791810076062, 18.171341177121942, 5.166606229006716, 1.1865180839783833, 0.1749432050895355, 0.005981435899831694, 0.014833866937032762, 0.03239755963539833, 0.03434485144540851, 0.027291313832892863, 0.01859294628158023, 0.011610942136174533, 0.006957703200806158, 0.004152248714722771, 0.0025382940645445264, 0.0016225791920779177, 0.0010890571995492676, 0.000759219589092473, 0.0005382575300630169, 0.00037939039651416537, 0.0002636979670620392, 0.00018144842843921703, 0.0001242931874868779, 8.406619410203045e-05, 5.502830224990328e-05, 3.49128621684375e-05, 2.2092097895480304e-05, 1.4715235573941155e-05, 1.0786975975891272e-05, 8.818923512623307e-06, 8.098393585963174e-06, 8.15162776663435e-06, 8.892603535772417e-06, 1.032772432217692e-05, 1.1946320841467074e-05, 1.308431845454934e-05, 1.3517844199378415e-05, 1.3808941509962294e-05, 1.452673440217328e-05, 1.5101089560407937e-05, 1.4925192559987416e-05, 1.4024645029618592e-05, 1.3223812615906297e-05, 1.2964227945174487e-05, 1.253618565228984e-05, 1.1462327617524668e-05, 1.0082127710727479e-05, 9.178507921659704e-06, 8.767163737919888e-06]
hSTA_Ba = [76315.492666811, 39812.823975959924, 20972.234062628817, 10966.61133496681, 5611.050583885593, 2806.4400430841347, 1373.3999555879798, 650.9990713614202, 298.8783901601414, 131.6123786055753, 55.454123941098125, 21.942052943637734, 7.97463638944068, 2.562326927199085, 0.6721629184123306, 0.11268957320653297, 0.005107103263774923, 0.01516074661433383, 0.03472535886802001, 0.041413366501391415, 0.03847894352554621, 0.03136357597631728, 0.023056768652735152, 0.015468866464832145, 0.009813806912019423, 0.006205962897109747, 0.004179487384958529, 0.002979570869486426, 0.002133697694085241, 0.0014838665348794356, 0.000994021071223441, 0.0006969716729318375, 0.0005788679949094692, 0.0005274229338406443, 0.0004549801603531247, 0.00034238057719563914, 0.0002160815940633957, 0.00013574305640947905, 0.00012090368929921652, 0.000131618015750155, 0.00013063112865313673, 0.00010247300353148635, 5.69145905082356e-05, 2.651401080441898e-05, 2.5154490203254006e-05, 3.81090787775531e-05, 4.761881133449785e-05, 4.100661348361088e-05, 2.099912570213713e-05, 5.5897132764080246e-06, 4.4879273963176235e-06, 1.2885579018056264e-05, 2.1629091614659617e-05, 2.101851413172076e-05, 1.1113472451049938e-05, 2.0840032381794676e-06]
hLuke_Ca = [0.050239421543304345, 0.0011883010378322119, 0.001146074464246958, 0.00047109949154036857, 0.00021027358615982635, 2.588130199563041e-05, 2.9890289838288458e-05, 1.2482465914422272e-05, 2.5658846439036653e-06, 1.1095829550235165e-05, 1.6892248058978703e-06, 5.954078440676725e-06, 4.197277031335132e-06, 3.708445959861838e-06, 5.545578199683238e-06, 3.4567267267839002e-06, 4.5936887096666685e-06, 3.976755027612137e-06, 4.282244126367543e-06, 4.6901224041281025e-06, 3.933670658608209e-06, 4.2584912814021035e-06, 4.079454160386876e-06, 4.415993903356353e-06, 4.465966173347172e-06, 4.057687679340536e-06, 4.148493314273964e-06, 4.18787111619973e-06, 4.464466023340308e-06, 4.355593366358331e-06, 4.066972289747402e-06, 4.125568365830138e-06, 4.277595702780372e-06, 4.464315411763072e-06, 4.284271425938602e-06, 4.053642072752238e-06, 3.9868424875250225e-06, 4.350236643670336e-06, 4.446073059199828e-06, 4.215271925995788e-06, 4.048195595717837e-06, 3.963149708718561e-06, 4.401653393024894e-06, 4.411598925371688e-06, 4.178138762163244e-06, 4.051587457362474e-06, 4.29676438124126e-06, 4.423169025441927e-06, 4.357287494569848e-06, 4.10259761034717e-06, 4.079759012150634e-06, 4.318798685351904e-06, 4.439782885660794e-06, 4.295120483878477e-06, 4.0633693449106075e-06, 4.125958065428658e-06]
hLuke_Sr = [17.62176841297568, 3.555192264288984, 0.09114516705547258, 0.13075387627397245, 0.001858539557272939, 0.00737344377539751, 0.0014121631428277294, 0.000632355275295701, 0.0003047024236990881, 9.170831882767541e-05, 4.355857952600631e-05, 1.3260192108848545e-05, 5.253762828315148e-06, 2.623860859932684e-06, 3.05639214509813e-06, 4.468338509138739e-06, 5.9172200307228426e-06, 6.750177310015465e-06, 6.8757689386223575e-06, 6.618658746073468e-06, 6.331979973287496e-06, 6.129907030697696e-06, 5.888086855834227e-06, 5.47209788976244e-06, 4.8975058029595805e-06, 4.286546595718161e-06, 3.818139692327384e-06, 3.606714163723782e-06, 3.6070913290307667e-06, 3.7179630607571954e-06, 3.797700257123591e-06, 3.7758504455578426e-06, 3.6651893073677846e-06, 3.5887699050046488e-06, 3.620200309108408e-06, 3.832992330134382e-06, 4.151015516368796e-06, 4.2779789527027225e-06, 4.268707279432715e-06, 4.1085365361230435e-06, 3.924006842515537e-06, 3.87827910999494e-06, 3.995730344041568e-06, 4.179160539843205e-06, 4.283774816895782e-06, 4.211108445663068e-06, 3.98482302533594e-06, 3.800561835795134e-06, 3.7344526638705643e-06, 3.838406915205114e-06, 4.046411100082276e-06, 4.174399365356435e-06, 4.145751998503282e-06, 3.985195978192105e-06, 3.8024078487106406e-06, 3.7496175695152876e-06]
hLuke_Ba = [336.487964646329, 40.541361565262434, 3.6050678645577103, 1.857518741142442, 0.048613934960111314, 0.12109277457390429, 0.0019913213837094166, 0.013072227190918547, 0.0009071822609174113, 0.0015809849939749949, 0.0005716006062487501, 0.0003863333714212533, 0.00011430828850226788, 8.968392739317605e-05, 8.29914064481968e-05, 4.185969912622637e-05, 9.388480518867437e-06, 9.28404690582321e-06, 1.8700964484298742e-05, 1.3780024375501733e-05, 2.3224883500850176e-06, 1.0567025057756616e-06, 6.885379682611582e-06, 7.434294049074912e-06, 2.8724610747408096e-06, 8.132369126114995e-07, 3.0201788187044705e-06, 5.00323449377878e-06, 3.869951740253376e-06, 2.054541572186871e-06, 2.596521538964943e-06, 3.953317792923243e-06, 3.6714944304840233e-06, 2.428516816012108e-06, 2.161283220929791e-06, 2.979960403428004e-06, 3.7005576516754478e-06, 3.166659603316446e-06, 2.757699657045445e-06, 3.0533266907203965e-06, 3.351049448448381e-06, 3.0539760432140324e-06, 2.574700468557521e-06, 2.5720272451340897e-06, 2.969526722274416e-06, 3.2012518944748585e-06, 3.1150298918364136e-06, 3.0134713728958093e-06, 3.12688711542498e-06, 3.110971885211733e-06, 2.871311471752831e-06, 2.6419924327000413e-06, 2.72048284352561e-06, 2.9760611294270144e-06, 3.1432907012294533e-06, 3.1396066456909873e-06]


Tf = np.linspace(5, 60, 56)
fig, ax = plt.subplots()
plt.ylabel('Phonon')
plt.xlabel('Split Time (μs)')
plt.semilogy(Tf, hSTA_Ca, marker='.', ls='--', color='tab:blue', label='40Ca STA')
plt.semilogy(Tf, hSTA_Sr, marker='.', ls='--', color='tab:orange', label='88Sr STA')
plt.semilogy(Tf, hSTA_Ba, marker='.', ls='--', color='tab:green', label='133Ba STA')
plt.semilogy(Tf, hLuke_Ca, marker='s', color='tab:blue', label='40Ca')
plt.semilogy(Tf, hLuke_Sr, marker='s', color='tab:orange', label='88Sr')
plt.semilogy(Tf, hLuke_Ba, marker='s', color='tab:green', label='133Ba')
plt.semilogy(Tf, h_T_Ca, marker='x', ls=':', color='tab:blue', label='40Ca')
plt.semilogy(Tf, h_T_Sr, marker='x', ls=':', color='tab:orange', label='88Sr')
plt.semilogy(Tf, h_T_Ba, marker='x', ls=':', color='tab:green', label='133Ba')

custom_lines = [Line2D([0], [0], color='tab:blue'), Line2D([0], [0], color='tab:orange'), Line2D([0], [0], color='tab:green')]
custom_lines2 = [Line2D([0], [0], color='tab:grey', ls=':', marker='x', lw=2), Line2D([0], [0], color='tab:grey', ls='--', marker='.', lw=2), Line2D([0], [0], color='tab:grey', ls='solid', marker='s', lw=2)]

ax.legend(custom_lines, ['40Ca', '88Sr', '133Ba'])
ax2 = ax.twinx()
ax2.legend(custom_lines2, ['Kaufmann', 'Palmero', 'Qi'], loc=(0.825, 0.6))

# plt.legend(loc='best', shadow=1)
plt.title('Comparisons Across Three Splitting Methods')
# plt.title('Ion Splitting: Inner vs Outer Segmentation')


In [ ]:
N = 2**16 ## whether or not you zero pad makes little difference
t_fft, dx = np.linspace(0, 100e-6, 4096, retstep=1)
q_w = np.fft.rfftfreq(N, dx)
q_w_func = interp1d(range(len(q_w)), q_w)
q_w_ifunc = interp1d(q_w, range(len(q_w)))

Tvec = np.array([n/10*1E-6 for n in range(10, 150)])
array = []
for T in Tvec:
#     q_t = np.array([qtrig(t, (40e-6, 40e-6+T), f=1.2e6, mu=1) for t in t_fft])
    q_t = np.array([qsta(t, (40e-6, 40e-6+T), f=1.2e6*T) for t in t_fft])
    q_t = np.gradient(q_t, dx)
    q_t = np.gradient(q_t, dx)*np.hanning(len(q_t)) ## whether or not you use a hanning window makes little difference
    q_fft = np.fft.rfft(q_t, N)
    q_fftmag = np.abs(q_fft)
    q_fftphs = np.angle(q_fft)
    array.append(np.log(q_fftmag+1e-20))
    
spec = np.array(array).T
title = 'Spectrogram of STA poly transport'
plt.figure(title.replace(' ', '_'))
plt.imshow(spec, aspect = 'auto', ## Prevent the image from getting stretched out
    interpolation = 'nearest', 
    origin = 'lower', ## Important to do this so that the first row goes on the bottom
    cmap = 'viridis',
    vmin = 0)
plt.axis([0, len(Tvec), 0, N/20])
plt.colorbar()
plt.show()
ticks = ticker.FuncFormatter(lambda x, pos: '{0:.1f}'.format(x/10+1))
plt.axes().xaxis.set_major_formatter(ticks)
ticks = ticker.FuncFormatter(lambda y, pos: '{0:.0f}'.format(q_w_func.__call__(y)))
plt.axes().yaxis.set_major_formatter(ticks)
plt.xlabel('Transport Time (us)')
plt.ylabel('Frequency Content (Hz)')
plt.title(title)

## Horizontal Slice
# title2 = 'Slice of Spectrogram at 1.2MHz'
# plt.figure(title2)
# plt.plot(Tvec*1e6, spec[int(q_w_ifunc.__call__(1.2e6))], marker='.', label='STA trig')
# plt.title(title2)
# plt.legend(loc='best', shadow=1)
# plt.xlabel('Transport Time (us)')
# plt.ylabel('FFT Magnitude (log)')

## Vertical Slice
# plt.figure('vertical slice')
# plt.plot(q_w, array[10]) ## T=2us
# plt.plot(q_w, array[130]) ## T=14us

